<a id='intro'></a>

# Tools for exploring ICOS Atmospheric data & STILT results


This notebook includes functions in Python for exploring ICOS data and STILT results. The notebook is divided in the following parts:

- [Import Python modules](#import_modules)
- [Set paths](#datapaths)
- [Data availability](#data_availability)
- [Map functions](#map_funcs)
- [Read STILT timeseries](#read_stilt_timeseries)
- [Plotting functions](#plotting_funcs)
- [Footprint functions](#footprint_funcs)
- [Widget functions](#widget_func)
- [Main function](#main)



Use the links to quickly navigate to the parts you are interested in. 

<a id='import_modules'></a>
<br>
<br>

## 1. Import modules
In this part, we are going to import all Python modules that we are going to use.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#Import modules:
import sys
import numpy as np
from numpy import nan
import pandas as pd
import pickle
import zipfile
import netCDF4 as cdf
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import fnmatch
import os
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.markers as markers

import folium
import branca

import cartopy
cartopy.config['data_dir'] = '/data/project/cartopy/'
from cartopy import config
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
import cartopy.feature as cfeature


from ipywidgets import interact, interact_manual, ColorPicker, Dropdown, SelectMultiple, Checkbox, DatePicker, SelectionRangeSlider
from bokeh.plotting import show
from bokeh.io import reset_output, output_notebook

#Import ICOS tools:
from icoscp.sparql import sparqls, runsparql
from icoscp.cpb.dobj import Dobj
from icoscp.station import station
from tools.check_funcs.numeric import is_int
from tools.country.fullname import get_country_fullname_from_iso3166_2char
from tools.visualization.string.format_output import printmd

#Set the notebook as the default output location:
reset_output()
output_notebook()


<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='datapaths'></a>

<br>
<br>

## 2. Set paths
In this part, we set the path to the location where the STILT output datafiles are stored:

In [ ]:
#Set paths to data:
path_stiltweb_stations = '/data/stiltweb/stations/'
path_fp = '/data/stiltweb/slots/'

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='data_availability'></a>

<br>
<br>

## 3. Data availability
This part includes functions that check for availability of:
- CO$_2$ atmospheric data per ICOS station (Level 1 & Level 2)
- STILT results per station
- STILT results per ICOS station (Level 1 & Level 2 CO$_2$ atmospheric data products) 

The functions above are used to populate an availability table.

In [ ]:
def get_stilt_stationID_samplHeight(stiltStInfo):
   
    """
    Project:         'ICOS Carbon Portal'
    Created:          Thu Apr 25 09:10:00 2019
    Last Changed:     Thu Apr 25 09:10:00 2019
    Version:          1.0.0
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes a string as input representing the STILT station ID (e.g. 'CES050').
                      The function discards the first 3 chatacters in the string 
                      (representing the station code) and checks the remaining part of the string; if the
                      remaining part contains digits, it returns the sampling height as a string,
                      otherwise it returns an empty string.
                      
    Input parameters: STILT station ID (var_name: 'stiltStInfo', var_type: String)

    Output:           Sampling Height (var_type: String)
    
    """
    
    if(stiltStInfo[3:len(stiltStInfo)].isdigit()):
        stSH = stiltStInfo[3:len(stiltStInfo)]
        return str(float(stSH))
        
    else:
        return ''
    

In [ ]:
def get_stilt_stations_df(pathStations):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 26 09:00:00 2019
    Last Changed:     Mon Oct 05 16:00:00 2020
    Version:          1.1.0
    Author(s):        Karolina Pantazatou
    
    Description:      Function that takes a string as input representing the path to the STILT station directories.
                      The function return discards the first 3 chatacters in the string 
                      (representing the station code) and checks the remaining part of the string; if the
                      remaining part contains digits, it converts the string to float and returns the 
                      sampling height as a numeric value, otherwise it returns an empty string.
                      
    Input parameters: Path to STILT output directory (var_name: 'pathStations', var_type: String)

    Output:           Pandas Dataframe
    
                      Columns: 
                      STILT station ID, e.g. 'GAT060' (var_name: 'stiltStationId', var_type: String)
                      STILT station 3-char code (var_name: 'stiltStation', var_type: String)
                      Sampling Height (var_name: 'stiltSamplingHeight', var_type: Float)
    
    """
    #Check if directory exists:
    if os.path.exists(pathStations):
        
        #Get a list of stations for existing STILT-runs:
        allStations = os.listdir(pathStations)

        #Get a list of tuples with unique occurances of station ID and
        #sampling height. Convert list of tuples to a pandas dataframe:
        stilt_stationID_df = pd.DataFrame(sorted(list(set([(allStations[i],
                                                            allStations[i][0:3],
                                                            get_stilt_stationID_samplHeight(allStations[i]))
                                                           for i in range(len(allStations))
                                                           if ((allStations[i][3:len(allStations[i])].isdigit())|
                                                               (allStations[i][3:len(allStations[i])]==''))]))),
                                          columns=['stiltStationId', 'stiltStation', 'stiltSamplingHeight'])
       
    else:
        print ('Directory '+pathStations+ ' does not exist!')
        
    #Return dataframe:
    return stilt_stationID_df

In [ ]:
def get_stilt_availability_df(pathStations, stilt_icos_availability_df, stilt_st_col):

    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Apr 30 09:30:00 2019
    Last Changed:     Sun Oct 11 09:30:00 2020
    Version:          1.1.0
    Author(s):        Karolina
    
    Description:      Function that takes three input parameters; a string containing the path to the
                      directory where the STILT model run outputs are stored, a dataframe with info
                      for ICOS atmospheric stations for which STILT model run results are available for
                      and string representing the column name for STILT station ID in that dataframe.
                      The function returns a pandas dataframe with three columns; STILTstationID, year,
                      month. Every row contains information regarding the year and the month for which
                      STILT data are available for per station.
                      
    Input parameters: 1. Path to directory containing STILT Model Results
                         (var_name: "pathStations", var_type: String)
                      2. ICOS Stations for which STILT Model Results are available for
                         (var_name: "stilt_icos_availability_df", var_type: Pandas Dataframe)
                      3. column name for STILT station ID in availability pandas dataframe
                         (var_name: "stilt_st_col", var_type: String)

    Output:           Pandas Dataframe
    
                      Columns:
                      1. STILT Station ID (var_name: "stiltStationId", var_type: String)
                      2. Year (var_name: "year", var_type: Integer)
                      3. Month for which STILT Model Results are available for (for a given year)
                        (var_name: "month", var_type: Integer)
    
    """
    
    #Create list of tuples to store availability of stilt results for icos stations expressed as
    #number of months per year per station:
    stilt_output_months_per_year_df = pd.concat([pd.DataFrame(sorted([(stilt_stationId, int(year),int(x))
                                                                      for x in os.listdir(path_stiltweb_stations+'/'+stilt_stationId+'/'+year)
                                                                      if len(x)<3]), columns=[stilt_st_col, 'year', 'month'])
                                                 for stilt_stationId
                                                 in sorted(stilt_icos_availability_df[stilt_st_col].unique())
                                                 for year
                                                 in sorted(os.listdir(path_stiltweb_stations+'/'+stilt_stationId))
                                                 if is_int(year)])

    #Drop duplicate rows and reset index:
    stilt_output_months_per_year_df = stilt_output_months_per_year_df.drop_duplicates().reset_index().drop('index', axis=1)
   

    
    #Return dataframe:
    return stilt_output_months_per_year_df

In [ ]:
def get_icos_availability_df(atm_stations, prod_list):

    #Get dataframe with ICOS station info 
    #for defined L1 & L2 Atmospheric Data Products:
    icos_df = pd.concat([atm_stations[j].data().loc[(atm_stations[j].data().specLabel==prod_list[0])|
                                                    (atm_stations[j].data().specLabel==prod_list[1])]
                         for j in range(len(atm_stations))])

    #Reset index:
    icos_df.reset_index(inplace=True)

    #Add column with ICOS station ID:
    icos_df['stationId'] = [icos_df.station.iloc[y][-3:] for y in range(len(icos_df))]

    #Drop old integer index:
    icos_df.drop(['index'], axis=1, inplace=True)

    #Return dataframe:
    return icos_df

In [ ]:
def get_icos_stilt_availability_df(path_stilt):
    
    #Get dataframe with STILT-info:
    stilt_df = get_stilt_stations_df(path_stilt)
    
    #Get dataframe with ICOS station info 
    #(for stations providing L1 & L2 CO2 Atmospheric Data Product):
    icos_df = get_icos_availability_df(atm_stations, ['ICOS ATC CO2 Release', 'ICOS ATC NRT CO2 growing time series'])
    
    #Create a dataframe with the STILT-station exceptions:
    exceptions_df = pd.DataFrame({'stiltStationId': ['GAT344', 'KIT050', 'LIN099', 'SMR127'],
                                  'stiltStation': ['GAT', 'KIT', 'LIN', 'SMR'],
                                  'stiltSamplingHeight': ['344.0', '50.0', '99.0', '127.0'],
                                  'icosSamplingHeight': ['341.0', '60.0', '98.0', '125.0']})

    #Do a cartesian product (natural join) between icos_df & stilt_df:
    icos_stilt = pd.merge(icos_df, stilt_df,
                          left_on='stationId',
                          right_on='stiltStation')

    #Get a pandas dataframe including ICOS stations for which stilt-output
    #is available for:
    availability_df = pd.concat([icos_stilt.loc[(icos_stilt.stationId==icos_stilt.stiltStation)&
                                                (icos_stilt.stiltSamplingHeight=='')],
                                 pd.merge(icos_df, stilt_df,
                                          left_on=['stationId', 'samplingheight'],
                                          right_on=['stiltStation', 'stiltSamplingHeight']),
                                 pd.merge(icos_df, exceptions_df,
                                          left_on=['stationId', 'samplingheight'],
                                          right_on=['stiltStation', 'icosSamplingHeight'])])


    #Delete help-column(from exceptions_df) with ICOS station sampl. height:
    availability_df.drop('icosSamplingHeight', axis=1, inplace=True)

    #Convert data type of columns with time-info from string to datetime:
    availability_df['timeStart'] = pd.to_datetime(availability_df['timeStart']) 
    availability_df['timeEnd'] = pd.to_datetime(availability_df['timeEnd']) 

    #Add column  with location Identifier info:
    availability_df['locIdent'] = [os.path.split(os.readlink(path_stiltweb_stations+station))[-1]
                                   for station in availability_df.stiltStationId.values]

    #Add station sampling height for all STILT-stations
    #(especially those that had '' as sampling height before):
    availability_df['stiltSamplingHeight']= [str(np.float(locIdent[-5:])) for locIdent in availability_df.locIdent]
    
    #Return dataframe:
    return availability_df

In [ ]:
#Function that returns a pandas dataframe with the total number
#of months per year observations from a specific station regarding
#a specific data product are available for:
def get_nmonths_per_year(station_availability_df, time_start_col, time_end_col, st_column):
    
    def get_months_per_year_per_file(stationID, t_start, t_end):
        
        #Import modules:
        import pandas as pd
        from dateutil.relativedelta import relativedelta
        
        #Create empty list to store tuples of: "stationID", "year", "month"
        #[('GAT132', 2016, 5)]
        ls = []
        
        #Set current date to start date:
        cur_date = t_start
        
        #Loop through all months within start and end dates:
        while cur_date < t_end+relativedelta(hours=1):
            
            #Add tuple with "stationID", current date "year" and "month" to list:
            ls.append((stationID, cur_date.year, cur_date.month))
            
            #Change current date to one month ahead:
            cur_date += relativedelta(months=1)
            
        return pd.DataFrame(ls, columns=[st_column, 'year', 'month'])
        
        
    #Return pandas dataframe: 
    return pd.concat([get_months_per_year_per_file(station_availability_df[st_column].iloc[i],
                                                   station_availability_df[time_start_col].iloc[i],
                                                   station_availability_df[time_end_col].iloc[i])
                      for i in range(len(station_availability_df))]).drop_duplicates().reset_index().drop('index', axis=1)

In [ ]:
def data_availability_per_station_m(stilt_icos_df, stilt_st_col, sampl_height_col, data_prod_col, data_product, time_start_col, time_end_col):
    
    #Get a dataframe including how many data files are available per data product and station:
    availability_data_prod = stilt_icos_df.loc[stilt_icos_df.specLabel==data_product].groupby([stilt_st_col,
                                                                                               sampl_height_col,
                                                                                               data_prod_col]).size()

    
    #Get list of dataframes with number of months per year observations
    #are available for per station:
    availability_nmomths_per_year_df = pd.concat([get_nmonths_per_year(stilt_icos_df[[stilt_st_col,
                                                                                      data_prod_col,
                                                                                      time_start_col,
                                                                                      time_end_col]].loc[(stilt_icos_df[data_prod_col]==data_product) &
                                                                                                         (stilt_icos_df[stilt_st_col]==availability_data_prod.index[i][0])].reset_index(),
                                                                       time_start_col,
                                                                       time_end_col,
                                                                       stilt_st_col)
                                                  for i in range(len(availability_data_prod))]).reset_index().drop('index', axis=1)


    

    #Return dataframe:
    return availability_nmomths_per_year_df                                      

In [ ]:
def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        
        #Import module:
        import time 
        
        return time.mktime(date.timetuple())
    
    s = sinceEpoch

    year = date.year
    startOfThisYear = datetime(year=year, month=1, day=1)
    startOfNextYear = datetime(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

In [ ]:
def get_availability_per_month_per_station(stilt_path, stilt_icos_df, source, stilt_st_col, sampl_height_col, data_prod_col, data_product, time_start_col, time_end_col):
    
    #If input dataset consists of ICOS observations:
    if(source=='icos'):
    
        #Call function to get data availability for given ICOS data product: 
        prod_availability_df = data_availability_per_station_m(stilt_icos_df,
                                                               stilt_st_col,
                                                               sampl_height_col,
                                                               data_prod_col,
                                                               data_product,
                                                               time_start_col,
                                                               time_end_col)
        
    #If input dataset is STILT:
    else:
        
        #Call function to get data availability for STILT data: 
        prod_availability_df = get_stilt_availability_df(stilt_path,
                                                         stilt_icos_df,
                                                         stilt_st_col)
    #Add column with date:
    prod_availability_df['date']= pd.to_datetime(prod_availability_df.year*10000+prod_availability_df.month*100+1,format='%Y%m%d')

    #Add column with decimal date:
    prod_availability_df['decimalDate'] = [toYearFraction(prod_availability_df.date.iloc[i])
                                           for i in range(len(prod_availability_df))]
      
        
    #Return dataframe:
    return prod_availability_df

In [ ]:
def compute_sets(path_stilt, stilt_icos_df, stilt_st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    icos_L1_availability_df2 = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[0],
                                                                      time_start_col,
                                                                      time_end_col)


    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    icos_L2_availability_df2 = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[1],
                                                                      time_start_col,
                                                                      time_end_col)

    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    stilt_months_per_year_df = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'stilt',
                                                                      stilt_st_col,
                                                                      '',
                                                                      '',
                                                                      '',
                                                                      '',
                                                                      '')


    #Merge availability-DFs for ICOS L1 & ICOS L2:
    icos_L1_L2_intersect_df = pd.merge(icos_L1_availability_df2,
                                       icos_L2_availability_df2,
                                       how='inner',
                                       on=list(icos_L2_availability_df2.columns.values), sort=True)
    
    #Merge availability-DFs for ICOS L1 & STILT:
    icos_L1_STILT_intersect_df = pd.merge(icos_L1_availability_df2,
                                          stilt_months_per_year_df,
                                          how='inner',
                                          on=list(icos_L1_availability_df2.columns.values), sort=True)

    #Merge availability-DFs for ICOS L2 & STILT:
    icos_L2_STILT_intersect_df = pd.merge(icos_L2_availability_df2,
                                          stilt_months_per_year_df,
                                          how='inner',
                                          on=list(icos_L2_availability_df2.columns.values), sort=True)
    
    
    #STILT, ICOS L1 & ICOS L2:
    stilt_icos_L1_L2_df2 = pd.merge(icos_L1_STILT_intersect_df,
                                    icos_L2_STILT_intersect_df,
                                    how='inner',
                                    on=list(icos_L2_STILT_intersect_df.columns.values), sort=True)

    #Set the combination of 'stiltStationId', 'date' as index:
    icos_L1_L2_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L1_STILT_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L2_STILT_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    stilt_months_per_year_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L1_availability_df2.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L2_availability_df2.set_index([stilt_st_col, 'date'], inplace=True)
    
    #STILT & ICOS L1:
    stilt_icos_L1_df2 = icos_L1_STILT_intersect_df[~icos_L1_STILT_intersect_df.isin(icos_L1_L2_intersect_df)].dropna().reset_index()

    #STILT & ICOS L2:
    stilt_icos_L2_df2 = icos_L2_STILT_intersect_df[~icos_L2_STILT_intersect_df.isin(icos_L1_L2_intersect_df)].dropna().reset_index()

    #STILT:
    stilt_df2 = stilt_months_per_year_df[(~stilt_months_per_year_df.isin(icos_L1_STILT_intersect_df))&
                                         (~stilt_months_per_year_df.isin(icos_L2_STILT_intersect_df))].dropna().reset_index()
    
    #ICOS L1 & ICOS L2:
    icos_L1_L2_df2 = icos_L1_L2_intersect_df[~icos_L1_L2_intersect_df.isin(stilt_months_per_year_df)].dropna().reset_index()

    #ICOS L1:
    icos_L1_df2 = icos_L1_availability_df2[(~icos_L1_availability_df2.isin(icos_L1_STILT_intersect_df))&
                                           (~icos_L1_availability_df2.isin(icos_L2_STILT_intersect_df))&
                                           (~icos_L1_availability_df2.isin(icos_L1_L2_intersect_df))].dropna().reset_index()
    #ICOS L2:
    icos_L2_df2 = icos_L2_availability_df2[(~icos_L2_availability_df2.isin(icos_L1_STILT_intersect_df))&
                                           (~icos_L2_availability_df2.isin(icos_L2_STILT_intersect_df))&
                                           (~icos_L2_availability_df2.isin(icos_L1_L2_intersect_df))].dropna().reset_index()

    #Return list of sets:
    return [stilt_icos_L2_df2, stilt_icos_L1_df2, icos_L1_df2, icos_L1_L2_df2, icos_L2_df2, stilt_icos_L1_L2_df2, stilt_df2]

In [ ]:
def update_availability_plot(stilt_icos_df, path_stilt, stilt_st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Import modules:
    from tools.visualization.availability import plot_attr, availability_plot

    #Create a new object of the class PlotAttr:
    avail_plot_attr = plot_attr.PlotAttr()

    #Provide list of tick-values for y-axis:
    avail_plot_attr.y_range=list(reversed(sorted((list(stilt_icos_df.reset_index().stiltStationId.unique())))))

    #Define plot title:
    avail_plot_attr.title_text = 'ICOS - STILT availability table'

    #Set x- & y-axis labels:
    avail_plot_attr.xaxis_label = 'Time'
    avail_plot_attr.yaxis_label = 'Station'

    #Provide dataframe column name for values that will be plotted on x- & y-axis:
    avail_plot_attr.col_name_xaxis = 'decimalDate'
    avail_plot_attr.col_name_yaxis = 'stiltStationId'
    
    #Get a list of dataframes with availability info for
    #every combination of ICOS L1, L2 & STILT data products
    #per station, year and month:
    data_set_ls = compute_sets(path_stilt,
                               stilt_icos_df,
                               stilt_st_col,
                               sampl_height_col,
                               data_prod_col,
                               data_product_ls,
                               time_start_col,
                               time_end_col)
    
    #Create a list with names for every layer:
    layer_ls = ['STILT & ICOS L2', 'STILT & ICOS L1',
                'ICOS L1', 'ICOS L1 & ICOS L2', 'ICOS L2',
                'STILT, ICOS L1 & L2', 'STILT']
    
    #
    t_list = [("Station", "stiltStationId"),
              ("Year", "year"),
              ("Month", "month")]
    
    #Call function to display plot:
    availability_plot.plot_table(data_set_ls, layer_ls, t_list, avail_plot_attr)

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='map_funcs'></a>

<br>
<br>

## 4. Map functions
This part includes functions that create a map of stations for which STILT results and ICOS CO$_2$ atmospheric data products are available for.


In [ ]:
#Function that returns a list of sampling heights
#for ICOS CO2 atmosphere data products:
def get_sampl_height_AS_CO2(station_code):
    
    #Import modules:
    import pandas as pd
    from icoscp.station import station
    
    #Get dataframe with available ICOS data products
    #for input station:
    #st_data_info = station.get(station_code).data()
    
    #Return list of sampling heights for ICOS CO2 atmosphere data products:
    return sorted(list(map(float, [st.data().samplingheight.loc[(st.data().specLabel=='ICOS ATC CO2 Release')|
                                                                (st.data().specLabel=='ICOS ATC NRT CO2 growing time series')].unique()
                                   for st in atm_stations if st.stationId==station_code][0])))



In [ ]:
def plotmap_stilt(stations_df, selected_station, basemap, d_icon='cloud', icon_col='orange'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Sun Sep 13 17:00:00 2020
    Last Changed:     Sun Sep 13 17:00:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a pandas dataframe with info about ICOS Stations 
                      (for wich STILT results are available for),
                      the 3-character long station code of a selected station, the basemap type, the
                      marker icon and the marker color as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: pandas dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)
                      3. Type of basemap (e.g. OSM or imagery)
                         (var_name: 'basemap', var_type: String)
                      4. Marker icon name (domain specific)
                         (var_name: 'd_icon', var_type: String)
                      5. Marker color (domain specific)
                         (var_name: 'icon_col', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    
    #Import modules:
    import folium
    import branca
    
    #Check what type of basemap is selected:
    if(basemap=='Imagery'):
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
                       zoom_start=15,
                       tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                       attr = 'Esri',
                       name = 'Esri Satellite',
                       overlay = False,
                       control = True)

    else:
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
                       zoom_start=15)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, st_dict, marker_txt, marker_color):
        
        #Add popup text:
        popup=folium.Popup(marker_txt, parse_html=True, max_width=400)
        
        #Create marker and add it to the map:
        folium.Marker(location=[float(st_dict['lat']),float(st_dict['lon'])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon=d_icon),
                      tooltip=tooltip).add_to(map_obj)



    #Create markers for all stations except selected station:
    for st in range(len(stations_df)):
        
        #Create and initialize variable to store station info in html table:
        html_table = """<meta content="text/html; charset=UTF-8">
                        <style>td{padding: 3px;}</style><table>"""
        
        for i in range(len(stations_df.columns)):
            
            #Check if column contains info about 'country':
            if(stations_df.columns[i]=='country'):
                
                #Add country name:
                html_table = html_table+'<tr><td>'+ stations_df.columns[i]+': </td><td><b>'+get_country_fullname_from_iso3166_2char(str(stations_df.iloc[st][stations_df.columns[i]]))+'</b></td></tr>'
            
            
            #Check if column contains info about station PI:
            elif((stations_df.columns[i]=='lastName')& (('firstName' in stations_df.columns)&('lastName' in stations_df.columns))):
                
                #Add project info:
                html_table = html_table+'<tr><td>station PI: </td><td><b>'+str(stations_df.iloc[st].firstName)+' '+str(stations_df.iloc[st].lastName)+'</b></td></tr>'
        
        
            #Check if dataframe contains info about station name and station uri:
            elif((stations_df.columns[i]=='name') & ('uri' in stations_df.columns)):
                
                #Add link to station landing page:
                html_table = html_table+'<tr><td>'+stations_df.columns[i]+'</td><td><b><a href="'+str(stations_df.iloc[st]['uri'])+'"target="_blank">'+str(stations_df.iloc[st][stations_df.columns[i]])+'</a></b></td></tr>'           
            
            #Skip columns:
            elif((stations_df.columns[i]=='firstName') | (stations_df.columns[i]=='uri')):
                continue
            
            else:
                
                #Add column info:
                html_table = html_table+'<tr><td>'+ stations_df.columns[i]+': </td><td><b>'+str(stations_df.iloc[st][stations_df.columns[i]])+'</b></td></tr>'
        
        #Add sampling height info:
        html_table = html_table+'<tr><td>sampling height: </td><td><b>'+', '.join(map(str, get_sampl_height_AS_CO2(stations_df.iloc[st].stationId)))+'</b></td></tr>'
        
        #Add html closing tag for table:
        html_table = html_table +'</table>'

        #Get station info in html-format and add it to an iframe:
        iframe = branca.element.IFrame(html=html_table, width=350, height=400)

        #Create dictionary with station lat/lon:
        st_loc_dict = {'lat':stations_df.iloc[st].lat,
                       'lon':stations_df.iloc[st].lon,}

        #Check if current station is selected station:
        if(stations_df.iloc[st].stationId==selected_station):
            
            #Add marker for selected station:
            add_marker(m, st_loc_dict, iframe, 'darkred')
   
        else:
            #Add marker for current station to map:
            add_marker(m, st_loc_dict, iframe, icon_col) 

    

    #Show map:
    display(m)



<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='read_stilt_timeseries'></a>

<br>
<br>

## 5. Read STILT timeseries
This part includes functions that read STILT timeseries and create list of tuples with ICOS-STILT station labels.

In [ ]:
#Function that returns labels for station-selection-widget:
def create_icos_stilt_labels(icos_stilt_df, station_info_df):

    #Filter by ICOS CO2 L2 data product:
    icos_stilt_L2_df = icos_stilt_df.loc[icos_stilt_df.specLabel=='ICOS ATC CO2 Release']
    
    #Get a list of tuples for every station that has provided tracer-data:
    #Every tuple is constructed like: ('Gartow (alt. 30.0)', ['GAT030', 'GAT', '30.0'])
    return sorted([(station_info_df.name.loc[station_info_df.stationId==icos_stilt_L2_df.stationId.iloc[i]].values[0]+ " (alt. " + str(icos_stilt_L2_df.stiltSamplingHeight.iloc[i]) + ")",
                    [icos_stilt_L2_df.stiltStationId.iloc[i],
                     str(icos_stilt_L2_df.stiltSamplingHeight.iloc[i]),
                     icos_stilt_L2_df.stationId.iloc[i],
                     str(icos_stilt_L2_df.samplingheight.iloc[i])])
                   for i in range(len(icos_stilt_L2_df))],
                  key = lambda x: (x[0].split(' (', 1)[0], float(x[1][1])))

In [ ]:
def read_stilt_timeseries(station, date_range, stilt_icos_availability_df, stilt_fp_path):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Oct 08 10:30:00 2018
    Last Changed:     Mon Oct 12 09:00:00 2020
    Version:          1.1.1
    Author(s):        Ute Karstens, Karolina Pantazatou
    
    Description:      Function that takes three input parameters; STILT Station ID, a dataframe of Datetime Objects
                      and a dataframe showing the STILT Model Result and ICOS Level-1 & Level-2 data availability,
                      checks if there are STILT concentration time series available for the given parameters, and, if
                      this is the case, returns the available STILT concentration time series in a pandas dataframe.
                      
    Input parameters: 1. STILT Station Code
                         (var_name: 'station', var_type: String)
                      2. Dataframe with one column of Datetime Objects
                         (var_name: 'date_range', var_type: Pandas Dataframe)
                      3. Dataframe with info about ICOS Atmospheric Stations with STILT results
                         (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)
                      4. Path to STILT time series files
                         (var_name: 'stilt_fp_path', var_type: String)

    Output:           Pandas Dataframe
                      
                      Columns:
                      1. Time (var_name: "isodate", var_type: date),
                      2. STILT CO2 (var_name: "co2.fuel", var_type: float)
                      3. Biospheric CO2 emissions (var_name: "co2.bio", var_type: float)
                      4. Background CO2 (var_name: "co2.background", var_type: float)
    
    """
    
    #Add URL:
    url = 'https://stilt.icos-cp.eu/viewer/stiltresult'
    
    #Add headers:
    headers = {'Content-Type': 'application/json', 'Accept-Charset': 'UTF-8'}
    
    #Create an empty list, to store the new time range with available STILT model results:
    new_range=[]
    
    #Loop through every Datetime object in the dataframe:
    for zDate in date_range:
        
        #Check if STILT results exist:
        if os.path.exists(stilt_fp_path +
                          stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==station].values[0]+
                          '/'+
                          str(zDate.year)+'/'+str(zDate.month).zfill(2)+'/'+
                          str(zDate.year)+'x'+str(zDate.month).zfill(2)+'x'+str(zDate.day).zfill(2)+'x'+
                          str(zDate.hour).zfill(2)+'/'):
            
            #If STILT-results exist for the current Datetime object, append current Datetime object to list: 
            new_range.append(zDate)
    
    #If the list is not empty:
    if len(new_range) > 0:
        
        #Assign the new time range to date_range:
        date_range = new_range
        
        #Get new starting date:
        fromDate = date_range[0].strftime('%Y-%m-%d')
        
        #Get new ending date:
        toDate = date_range[-1].strftime('%Y-%m-%d')
        
        #Store the STILT result column names to a variable:
        columns = ('["isodate","co2.stilt","co2.fuel","co2.bio", "co2.background"]')
        #columns = ('["isodate","co2.stilt","co2.fuel","co2.bio","co2.fuel.coal","co2.fuel.oil",'+
                   #'"co2.fuel.gas","co2.fuel.bio","co2.energy","co2.transport", "co2.industry",'+
                   #'"co2.others", "co2.cement", "co2.background",'+
                   #'"co.stilt","co.fuel","co.bio","co.fuel.coal","co.fuel.oil",'+
                   #'"co.fuel.gas","co.fuel.bio","co.energy","co.transport", "co.industry",'+
                   #'"co.others", "co.cement", "co.background",'+
                   #'"rn", "rn.era","rn.noah","wind.dir","wind.u","wind.v","latstart","lonstart"]')
        
        #Store the STILT result data column names to a variable:
        data = '{"columns": '+columns+', "fromDate": "'+fromDate+'", "toDate": "'+toDate+'", "stationId": "'+station+'"}'
        
        #Send request to get STILT results:
        response = requests.post(url, headers=headers, data=data)
        
        #Check if response is successful: 
        if response.status_code != 500:
            
            #Get response in json-format and read it in to a numpy array:
            output=np.asarray(response.json())
            
            #Convert numpy array with STILT results to a pandas dataframe: 
            df = pd.DataFrame(output[:,:], columns=eval(columns))
            
            #Replace 'null'-values with numpy NaN-values:
            df = df.replace('null',np.NaN)
            
            #Set dataframe data type to float:
            df = df.astype(float)
            
            #Convert the data type of the 'date'-column to Datetime Object:
            df['date'] = pd.to_datetime(df['isodate'], unit='s')
            
            #Set 'date'-column as index:
            df.set_index(['date'],inplace=True)
            
            #Add column with station name:
            df['name'] = station
            
            #Add column with model - name:
            df['model'] = 'STILT'
            
        else:
            
            #Print message:
            print("\033[0;31;1m Error...\nToo big STILT dataset!\nSelect data for a shorter time period.\n\n")
            
            #Create an empty dataframe:
            df=pd.DataFrame({'A' : []})
            
            
    #If the request fails, return an empty dataframe:        
    else:
        df=pd.DataFrame({'A' : []})
        
    #Return dataframe:    
    return df

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='plotting_funcs'></a>

<br>
<br>

## 6. Plotting functions
This part includes functions that create interactive plots with ICOS CO$_2$ atmospheric data products (Level 1 and Level 2) and STILT results.

In [ ]:
def plot_icos_single_station_binary(df_data, station_info_dict, tracer_info_dict, level=2, color='#0F0C08'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 07 09:30:00 2019
    Last Changed:     Mon Apr 07 09:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that plots the content of an ICOS Level-1 or Level-2 Atmospheric Data file to an 
                      interactive plot using the Bokeh interactive visualization library.
                      Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. ICOS Level-1 or Level-2 Atmospheric Observation Data
                         (var_name: 'df_data', var_type: pandas dataframe) 
                      2. Dictionary with station info
                         (var_name: 'station_info_dict', var_type: dictionary) 
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: dictionary) 
                      4. Data level [optional]
                         (var_name: 'level', var_type: Integer)
                      5. Color for Line- or Circle Glyph [optional]
                         (var_name: 'color', var_type: String)
                     
    
    Default value for color: The default value for line-glyph or circle-glyph color is "lightblue".  
    
    Default value for level: The default value for data level is "2". Function calls for Level-2 data do not have
                             to include a value for the level input parameter. 
    
    Output:           Bokeh Figure Object (plot) 
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure, show
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = pd.to_datetime(df_data['TIMESTAMP'], unit='ms')
    y = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].values #give tracer as parameter, where tracer can be ['co', 'co2']
    z = df_data["NbPoints"].values
    w = df_data["Stdev"].values
    o = df_data["Flag"].values
    #u = df_data["InstrumentId"].values

    #Create a ColumnDataSource object:
    source = ColumnDataSource( data = {'x':x, 'y':y, 'z':z, 'w':w, 'o':o,} )

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+ ' (' +
               tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'    '+
               station_info_dict['station_name'] +', '+
               station_info_dict['station_country'] +', '+
               station_info_dict['station_sampling_height'] +' m.a.g.l.',
               tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

    #Create glyphs:
    g0 = p.circle('x','y', source=source, radius=.02, color=color)
    
    #If data is level-2 data:
    if(level==2):
        g1 = p.line('x','y', source=source, line_width=1, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))
    
    #If data is level-1 data:
    else:
        g1 = p.line('x','y', source=source, line_width=2, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),
                    line_dash='dotted')
      
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ('St dev', '@w{0.f}'),
        ('NbPoints', '@z'),
        ('Flag', '@o')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    


    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')

    #return plot:
    return p

In [ ]:
def plot_icos_single_station_L1_L2_binary(data_df_list, station_info_dict, tracer_info_dict, color='#0F0C08'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that returns an interactive Bokeh plot of ICOS Level-1 and Level-2
                      Atmopsheric Data for a given ICOS Atmosphere Station.
                      
    Input parameters: 1. List of metadata dataframes for every ICOS Level-1 or Level-2 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'metadata_df_list', var_type: List of pandas dataframes)
                      2. List of data dataframes for every ICOS Level-1 or Level-2 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_df_list', var_type: List of pandas dataframes)
                      3. Tracer/gas (var_name: "tracer", var_type: String)
                      4. Plot line/point color (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure, show
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' (' +tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' - Continuous air ( '+
               station_info_dict['station_code']+', '+#.encode('latin1').decode('utf8')+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m agl)' ,
               tools='pan,box_zoom,wheel_zoom,reset,save')
    
    
    #Create an empty list that will store the legend info:
    legend_it = []

    #Extract time and tracer values for every data level:
    x1 = data_df_list[0].index.values
    y1 = data_df_list[0][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values
    x2 = data_df_list[1].index.values
    y2 = data_df_list[1][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values

    #Create a circle and line glyph for the values of every emission category:
    r0 = p.circle(x1, y1, radius=.12, color=color, alpha=0.5)
    r1 = p.line(x1, y1, line_width=2, line_dash='dotted', line_alpha=0.5, color=color, name='L1, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    r2 = p.circle(x2, y2, radius=.12, color=color)
    r3 = p.line(x2, y2, line_width=1, color=color, name='L2, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 1 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r1]))
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 2 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r3]))



    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')
    
    #Create a label object to store the copyright text:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add copyright-label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p
    

In [ ]:
def plot_icos_L1_L2_stilt_single_station_binary(stilt_df, data_df_list, station_info_dict, tracer_info_dict):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that returns an interactive Bokeh plot of ICOS Level-1 and Level-2
                      Atmopsheric Data in conjunction with STILT Model Results for the same station.
                      
    Input parameters: 1. Dataframe with STILT Model Results for a specific ICOS station and timeperiod
                         (var_name: 'stilt_df', var_type: Pandas DataFrame)
                      2. List of data dataframes for every ICOS Level-1 or Level-2 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_df_list', var_type: List of pandas dataframes)
                      4. Dictionary with ICOS station info
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      5. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: Dictionary)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    from bokeh.plotting import figure, show
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker
    from datetime import datetime
    from tools.visualization.bokeh_help_funcs.secondary_yaxis import set_yranges_2y
    from tools.math.roundfuncs import rounddown_10, rounddown_20, rounddown_100, roundup_10, roundup_20, roundup_100

    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x1 = stilt_df.index.values                 #STILT time
    y1 = stilt_df['co2.stilt'].values          #STILT CO2
    y2 = stilt_df['co2.background'].values     #STILT background
    y3 = stilt_df['co2.fuel'].values           #STILT Fuel
    y4 = stilt_df['co2.bio'].values            #STILT bio
    x2 = data_df_list[0].index.values          #ICOS Level-1 time
    y5 = data_df_list[0][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]  #ICOS Level-1 CO2 observations
    x3 = data_df_list[1].index.values          #ICOS Level-2 time
    y6 = data_df_list[1][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]  #ICOS Level-2 CO2 observations
    
    

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (ppm)',
               x_axis_type='datetime',
               title = 'STILT Model Output ('+
               station_info_dict['station_code']+', sampling height: '+
               station_info_dict['station_sampling_height']+' m agl)',
               tools='pan,box_zoom,wheel_zoom,undo,reset,save')
    
    
    #Get primary and secondary y_axis limits:
    y1_stilt_min = (stilt_df[[ 'co2.stilt', 'co2.background']].min()).min()
    y1_stilt_max = (stilt_df[[ 'co2.stilt', 'co2.background']].max()).max()
    y1_icos_L1_min = (min(y5))
    y1_icos_L1_max = (max(y5))
    y1_icos_L2_min = (min(y6))
    y1_icos_L2_max = (max(y6))
    y1_axis_min = min([y1_stilt_min, y1_icos_L1_min, y1_icos_L2_min])
    y1_axis_max = max([y1_stilt_max, y1_icos_L1_max, y1_icos_L2_max])
    y2_axis_min = (stilt_df[[ 'co2.fuel', 'co2.bio']].min()).min()
    y2_axis_max = (stilt_df[[ 'co2.fuel', 'co2.bio']].max()).max()
    
    
    ##### Control min/max values for both y-axes to allign their major ticks #####
    #If the difference between min & max values is >120 for both axes:
    if(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = ticker1
        
    #If the difference between min & max values is >120 for the primary y-axis only:
    elif(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)<120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = SingleIntervalTicker(interval=20)
        
        
    #If the difference between min & max values is >120 for the secondary y-axis only:
    elif(((y1_axis_max-y1_axis_min)<120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=100)
        
    
    #If the difference between min & max values is <120 for both y-axes:
    else:
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=20)
    
    ######
    
    

    #Create glyphs:
    g0 = p.circle(x1, y1, radius=.02, color='blue')
    g1 = p.line(x1, y1, line_width=1.5, color='blue', name='STILT')
    g2 = p.circle(x1, y2, radius=.02, color='#42bdff')#'#6baed6'
    g3 = p.line(x1, y2, line_width=1.5, color='#42bdff', name='background')
    g4 = p.circle(x1, y3, radius=.02, color='#d01c8b', y_range_name='Yaxis2')
    g5 = p.line(x1, y3, line_width=1.5, color='#d01c8b', name='fuel', y_range_name='Yaxis2')
    g6 = p.circle(x1, y4, radius=.02, color='#4dac26', y_range_name='Yaxis2')
    g7 = p.line(x1, y4, line_width=1.5, color='#4dac26', name='bio', y_range_name='Yaxis2')
    g8 = p.circle(x2, y5, radius=.02, color='#0F0C08')
    g9 = p.line(x2, y5, line_width=1.5, line_dash='dotted', line_alpha=0.5, color='#0F0C08', name='ICOS L1 Obs')
    g10 = p.circle(x3, y6, radius=.02, color='#0F0C08')
    g11 = p.line(x3, y6, line_width=1.5, color='#0F0C08', name='ICOS L2 Obs')
    
      
    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((g1.name, [g0, g1]))
    legend_it.append((g3.name, [g2, g3]))
    legend_it.append((g5.name, [g4, g5]))
    legend_it.append((g7.name, [g6, g7]))
    legend_it.append((g9.name, [g8, g9]))
    legend_it.append((g11.name, [g10, g11]))
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ('Category', '$name')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units
    
    #Set primary y-axis ticker:
    p.yaxis.ticker = ticker1
    
    #Create 2nd y-axis: 
    bg_yaxis = LinearAxis(y_range_name='Yaxis2',
                          axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' Components (ppm): Bio & Fuel',
                          ticker=ticker2,
                          axis_label_standoff = 15)
    
    #Add secondary y-axis to the right:
    p.add_layout(bg_yaxis, 'right')
    

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')
    
    #Return plot:
    return p
    
    

In [ ]:
def plot_stilt_icos_single_station_binary(data_Lx_df, station_info_dict, tracer_info_dict, icos_data_level, stilt_df):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:00:00 2018
    Last Changed:     Tue May 07 10:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that returns an interactive Bokeh plot of ICOS Level-1 or Level-2
                      Atmopsheric Data for a given ICOS Atmosphere Station in conjunction with STILT
                      Model Results for the same station.
                      
    Input parameters: 1. Data dataframes with ICOS Level-1 or Level-2 Data
                         (var_name: 'data_Lx_df', var_type: Pandas DataFrame)
                      2. Dictionary with ICOS station info
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: Dictionary)
                      4. ICOS Data Level (var_name: "icos_data_level", var_type: Integer)
                      5. Dataframe with STILT Model Results for a specific ICOS station and timeperiod
                         (var_name: 'stilt_df', var_type: Pandas DataFrame)

    Output:           Bokeh Plot
    
    """

    #Import modules to create figure:
    from bokeh.plotting import figure, show
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker
    from datetime import datetime
    from tools.visualization.bokeh_help_funcs.secondary_yaxis import set_yranges_2y
    from tools.math.roundfuncs import rounddown_10, rounddown_20, rounddown_100, roundup_10, roundup_20, roundup_100

    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x1 = stilt_df.index.values                 #STILT time
    y1 = stilt_df['co2.stilt'].values          #STILT CO2
    y2 = stilt_df['co2.background'].values     #STILT background
    y3 = stilt_df['co2.fuel'].values           #STILT Fuel
    y4 = stilt_df['co2.bio'].values            #STILT bio
    x2 = data_Lx_df.index.values               #ICOS time
    y5 = data_Lx_df[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()] #ICOS CO2 observations
    

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (ppm)',
               x_axis_type='datetime',
               title = 'STILT Model Output ('+
               station_info_dict['station_code']+', sampling height: '+
               station_info_dict['station_sampling_height']+' m agl)',
               tools='pan,box_zoom,wheel_zoom,undo,reset,save')
    
    
    #Get primary and secondary y_axis limits:
    y1_stilt_min = (stilt_df[[ 'co2.stilt', 'co2.background']].min()).min()
    y1_stilt_max = (stilt_df[[ 'co2.stilt', 'co2.background']].max()).max()
    y1_icos_min = min(y5)
    y1_icos_max = max(y5)
    y1_axis_min = min([y1_stilt_min, y1_icos_min])
    y1_axis_max = max([y1_stilt_max, y1_icos_max])
    y2_axis_min = (stilt_df[[ 'co2.fuel', 'co2.bio']].min()).min()
    y2_axis_max = (stilt_df[[ 'co2.fuel', 'co2.bio']].max()).max()
    
    
    ##### Control min/max values for both y-axes to allign their major ticks #####
    #If the difference between min & max values is >120 for both axes:
    if(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = ticker1
        
    #If the difference between min & max values is >120 for the primary y-axis only:
    elif(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)<120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = SingleIntervalTicker(interval=20)
        
        
    #If the difference between min & max values is >120 for the secondary y-axis only:
    elif(((y1_axis_max-y1_axis_min)<120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=100)
        
    
    #If the difference between min & max values is <120 for both y-axes:
    else:
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=20)
    
    ######
    
    
    
    
    #Create glyphs:
    g0 = p.circle(x1, y1, radius=.02, color='blue')
    g1 = p.line(x1, y1, line_width=1.5, color='blue', name='STILT')
    g2 = p.circle(x1, y2, radius=.02, color='#42bdff')#'#6baed6'
    g3 = p.line(x1, y2, line_width=1.5, color='#42bdff', name='background')
    g4 = p.circle(x1, y3, radius=.02, color='#d01c8b', y_range_name='Yaxis2')
    g5 = p.line(x1, y3, line_width=1.5, color='#d01c8b', name='fuel', y_range_name='Yaxis2')
    g6 = p.circle(x1, y4, radius=.02, color='#4dac26', y_range_name='Yaxis2')
    g7 = p.line(x1, y4, line_width=1.5, color='#4dac26', name='bio', y_range_name='Yaxis2')
    g8 = p.circle(x2, y5, radius=.02, color='#0F0C08')
    g9 = p.line(x2, y5, line_width=1.5, color='#0F0C08', name='ICOS L'+str(icos_data_level)+' Obs')
    
      
    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((g1.name, [g0, g1]))
    legend_it.append((g3.name, [g2, g3]))
    legend_it.append((g5.name, [g4, g5]))
    legend_it.append((g7.name, [g6, g7]))
    legend_it.append((g9.name, [g8, g9]))
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ('Category', '$name')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units
    
    #Set primary y-axis ticker:
    p.yaxis.ticker = ticker1
    
    #Create 2nd y-axis: 
    bg_yaxis = LinearAxis(y_range_name='Yaxis2',
                          axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' Components (ppm): Bio & Fuel',
                          ticker=ticker2,
                          axis_label_standoff = 15)
    
    #Add secondary y-axis to the right:
    p.add_layout(bg_yaxis, 'right')
    

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')
    
    #Return plot:
    return p
  

In [ ]:
def plot_stilt_single_station_binary(stilt_df, Station, tracer):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 08 15:00:00 2019
    Last Changed:     Wed May 08 15:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that plots the content of an STILT model output data file to an 
                      interactive plot using the Bokeh interactive visualization library.
                      Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. Dataframe with STILT Model Result for a specific ICOS Station
                         (var_name: 'stilt_df', var_type: pandas dataframe)
                      2. List with ICOS Station Info
                         (var_name: "Station", var_type: List of Strings)
                      3. Name of gas/tracer - e.g. 'co2'
                         (var_name: 'tracer', var_type: String)

    Output:           Bokeh Figure Object (plot) 
    
    """
    
    #Import modules to create figure:
    from bokeh.plotting import figure, show
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker
    from datetime import datetime
    from tools.visualization.bokeh_help_funcs.secondary_yaxis import set_yranges_2y
    from tools.math.roundfuncs import rounddown_10, rounddown_20, rounddown_100, roundup_10, roundup_20, roundup_100

    #Dictionary for subscript/superscript transformations:
    #SUB = {ord(c): ord(t) for c, t in zip(u"0123456789", u"₀₁₂₃₄₅₆₇₈₉")}
    #SUP = {ord(c): ord(t) for c, t in zip(u"0123456789", u"⁰¹²³⁴⁵⁶⁷⁸⁹")}

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Define Datasets:
    x = stilt_df.index.values
    y1 = stilt_df['co2.stilt'].values
    y2= stilt_df['co2.background'].values
    y3 = stilt_df['co2.fuel'].values
    y4 = stilt_df['co2.bio'].values

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer.upper().translate(SUB) + ' (ppm)',
               x_axis_type='datetime',
               title = 'STILT Model Output ('+
               Station[0]+', sampling height: '+
               Station[1]+' m agl)',
               tools='pan,box_zoom,wheel_zoom,undo,reset,save')
    
    
    
    #Get primary and secondary y_axis limits:
    y1_axis_min = (stilt_df[[ 'co2.stilt', 'co2.background']].min()).min()
    y1_axis_max = (stilt_df[[ 'co2.stilt', 'co2.background']].max()).max()
    y2_axis_min = (stilt_df[[ 'co2.fuel', 'co2.bio']].min()).min()
    y2_axis_max = (stilt_df[[ 'co2.fuel', 'co2.bio']].max()).max()
    
    
    ##### Control min/max values for both y-axes to allign their major ticks #####
    #If the difference between min & max values is >120 for both axes:
    if(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = ticker1
        
    #If the difference between min & max values is >120 for the primary y-axis only:
    elif(((y1_axis_max-y1_axis_min)>120) & ((y2_axis_max-y2_axis_min)<120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min)
        y1_axis_max = roundup_100(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     100.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=100)
        ticker2 = SingleIntervalTicker(interval=20)
        
        
    #If the difference between min & max values is >120 for the secondary y-axis only:
    elif(((y1_axis_max-y1_axis_min)<120) & ((y2_axis_max-y2_axis_min)>120)):
        
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_100(y2_axis_min)
        y2_axis_max = roundup_100(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     100.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=100)
        
    
    #If the difference between min & max values is <120 for both y-axes:
    else:
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_20(y1_axis_min)
        y1_axis_max = roundup_20(y1_axis_max)
        y2_axis_min = rounddown_20(y2_axis_min)
        y2_axis_max = roundup_20(y2_axis_max)
        
        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min,
                                                     y1_axis_max,
                                                     y2_axis_min,
                                                     y2_axis_max,
                                                     20.0,
                                                     20.0, 
                                                     'Yaxis2')
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=20)
        ticker2 = SingleIntervalTicker(interval=20)
    
    ######
    
    
    #Create glyphs:
    g0 = p.circle(x, y1, radius=.02, color='blue')
    g1 = p.line(x, y1, line_width=1.5, color='blue', name='STILT')
    g2 = p.circle(x, y2, radius=.02, color='#8fd8ff')
    g3 = p.line(x, y2, line_width=1.5, color='#8fd8ff', name='background') ##42bdff
    g4 = p.circle(x, y3, radius=.02, color='#d01c8b', y_range_name='Yaxis2')
    g5 = p.line(x, y3, line_width=1.5, color='#d01c8b', name='fuel', y_range_name='Yaxis2')
    g6 = p.circle(x, y4, radius=.02, color='#4dac26', y_range_name='Yaxis2')
    g7 = p.line(x, y4, line_width=1.5, color='#4dac26', name='bio', y_range_name='Yaxis2')
      
    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((g1.name, [g0, g1]))
    legend_it.append((g3.name, [g2, g3]))
    legend_it.append((g5.name, [g4, g5]))
    legend_it.append((g7.name, [g6, g7]))
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer.upper().translate(SUB),'@y{0.f}'),
        ('Category', '$name')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units
    
    #Set primary y-axis ticker:
    p.yaxis.ticker = ticker1
    
    #Create 2nd y-axis: 
    bg_yaxis = LinearAxis(y_range_name='Yaxis2',
                          axis_label=tracer.upper().translate(SUB)+' Components (ppm): Bio & Fuel',
                          ticker=ticker2,
                          axis_label_standoff = 15)
    
    #Add secondary y-axis to the right:
    p.add_layout(bg_yaxis, 'right')
    

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #Define output location:
    output_notebook()

    #Show plot
    show(p)

In [ ]:
def update_icos_L1_L2_stilt_single_station_plot_binary(stilt_df, data_obj_id_L1_ls, data_obj_id_L2_ls, station,
                                                       tracer, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 08 15:00:00 2019
    Last Changed:     Wed May 08 15:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that updates the content of an interactive Bokeh plot with STILT Model Results,
                      ICOS Level-1 and ICOS Level-2 data. The function gets the data object IDs of the new ICOS
                      data objects and calls functions to read their corresponding data files. Then the ICOS data
                      are filtered to only include information for the selected dates. The dataframes with the new
                      STILT and ICOS data are then sent to a plot function. The plot function returns a Bokeh 
                      Figure Object which is plotted. The interactive plot was developed using the Bokeh 
                      interactive visualization library. Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. Dataframe with STILT Model Result for a specific ICOS Station
                         (var_name: 'stilt_df', var_type: pandas dataframe)
                      2. List of data dataframes for every ICOS Level-1 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L1_ls', var_type: List of pandas dataframes)
                      3. List of data dataframes for every ICOS Level-2 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L2_ls', var_type: List of pandas dataframes)
                      4. List with ICOS Station Info
                         (var_name: "station", var_type: List of Strings)
                      5. Tracer/gas (var_name: "tracer", var_type: String)
                      6. Start Date (var_name: "start_date", var_type: DateTime Object)
                      7. End Date (var_name: "end_date", var_type: DateTime Object)

    Output:           Bokeh Plot 
    
    """
    
    #import modules:
    import pandas as pd
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_L1_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    
    #Loop through station obj list:
    for st in atm_stations:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station[2]:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station[2]
            station_info_dict['station_sampling_height'] = station[3]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    #Create list to store the data dataframes of all data object IDs:
    L1_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L1 in data_obj_id_L1_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L1 = Dobj(dobjid_L1).get()
        
        #Add data dataframe of the current data object ID to the list:
        L1_data_df_ls.append(obs_data_df_L1)
        
    #Concatenate data dataframes to one dataframe:
    data_L1_df = pd.concat(L1_data_df_ls)
    
    #Add column with datetime object:
    data_L1_df['DateTime'] = pd.to_datetime(data_L1_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L1 = data_L1_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_L1.sort_index(inplace=True)
    
    #Filter dataframe to only contain data for the selected year:
    data_L1_df_filt = data_df_ind_L1.loc[start_date:end_date]
        
    
    #Create list to store the data dataframes of all data object IDs:
    L2_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L2 in data_obj_id_L2_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L2 = Dobj(dobjid_L2).get()
        
        #Add data dataframe of the current data object ID to the list:
        L2_data_df_ls.append(obs_data_df_L2)
        
    #Concatenate data dataframes to one dataframe:
    data_L2_df = pd.concat(L2_data_df_ls)
    
    #Add column with datetime object:
    data_L2_df['DateTime'] = pd.to_datetime(data_L2_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L2 = data_L2_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_L2.sort_index(inplace=True)
    
    #Filter dataframe to only contain data for the selected year:
    data_L2_df_filt = data_df_ind_L2.loc[start_date:end_date]
       
        
    #Plot station:
    p = plot_icos_L1_L2_stilt_single_station_binary(stilt_df,
                                                    [data_L1_df_filt, data_L2_df_filt],
                                                    station_info_dict,
                                                    tracer_info_dict)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p)

In [ ]:
def update_icos_single_station_plot_L1_L2_binary(data_obj_id_L1_ls, data_obj_id_L2_ls, station_code, station_sampl_height, tracer, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 08 15:00:00 2019
    Last Changed:     Wed May 08 15:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that updates the content of an interactive Bokeh plot with ICOS Level-1 and
                      ICOS Level-2 data. The function gets the data object IDs of the new ICOS data objects
                      and calls functions to read their corresponding data files. Then the ICOS data are
                      filtered to only include information for the selected dates. The dataframes with the new
                      ICOS data are then sent to a plot function. The plot function returns a Bokeh 
                      Figure Object which is plotted. The interactive plot was developed using the Bokeh 
                      interactive visualization library. Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. List of data dataframes for every ICOS Level-1 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L1_ls', var_type: List of pandas dataframes)
                      2. List of data dataframes for every ICOS Level-2 Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L2_ls', var_type: List of pandas dataframes)
                      3. Tracer/gas (var_name: "tracer", var_type: String)
                      4. Start Date (var_name: "start_date", var_type: DateTime Object)
                      5. End Date (var_name: "end_date", var_type: DateTime Object)

    Output:           Bokeh Plot 
    
    """
    
    #import modules:
    import pandas as pd
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_L1_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    #Loop through station obj list:
    for st in atm_stations:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station_code:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station_code
            station_info_dict['station_sampling_height'] = station_sampl_height
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    #Create list to store the data dataframes of all data object IDs:
    L1_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L1 in data_obj_id_L1_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L1 = Dobj(dobjid_L1).get()
        
        #Add data dataframe of the current data object ID to the list:
        L1_data_df_ls.append(obs_data_df_L1)
        
    #Concatenate data dataframes to one dataframe:
    data_L1_df = pd.concat(L1_data_df_ls)
    
    #Add column with datetime object:
    data_L1_df['DateTime'] = pd.to_datetime(data_L1_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L1 = data_L1_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_L1.sort_index(inplace=True)
    
    #Filter dataframe to only contain data for the selected year:
    data_L1_df_filt = data_df_ind_L1.loc[start_date:end_date]
        
        
    
    #Create list to store the data dataframes of all data object IDs:
    L2_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L2 in data_obj_id_L2_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L2 = Dobj(dobjid_L2).get()
        
        #Add data dataframe of the current data object ID to the list:
        L2_data_df_ls.append(obs_data_df_L2)
        
    #Concatenate data dataframes to one dataframe:
    data_L2_df = pd.concat(L2_data_df_ls)
    
    #Add column with datetime object:
    data_L2_df['DateTime'] = pd.to_datetime(data_L2_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L2 = data_L2_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_L2.sort_index(inplace=True)
    
    #Filter dataframe to only contain data for the selected year:
    data_L2_df_filt = data_df_ind_L2.loc[start_date:end_date]
    

        
    #Plot station:
    p = plot_icos_single_station_L1_L2_binary([data_L1_df_filt, data_L2_df_filt],
                                              station_info_dict,
                                              tracer_info_dict)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p)
   

In [ ]:
def update_icos_stilt_single_station_plot_binary(data_obj_id_Lx_ls, station, tracer, icos_data_level,
                                                 stilt_df, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 08 15:00:00 2019
    Last Changed:     Wed Nov 18 20:45:00 2020
    Version:          0.1.3
    Author(s):        Karolina Pantazatou
    
    Description:      Function that updates the content of an interactive Bokeh plot with STILT Model Results and
                      ICOS Level-1 or ICOS Level-2 data. The function gets the data object IDs of the new ICOS
                      data objects and calls functions to read their corresponding data files. Then the ICOS data
                      are filtered to only include information for the selected dates. The dataframes with the new
                      STILT and ICOS data are then sent to a plot function. The plot function returns a Bokeh 
                      Figure Object which is plotted. The interactive plot was developed using the Bokeh 
                      interactive visualization library. Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. List of data dataframes for every ICOS Level-x Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L1_ls', var_type: List of pandas dataframes)
                      2. List with ICOS Station Info
                         (var_name: "station", var_type: List of Strings)
                      3. Tracer/gas (var_name: "tracer", var_type: String)
                      4. ICOS Data Level (var_name: "icos_data_level", var_type: Integer)
                      5. Dataframe with STILT Model Result for a specific ICOS Station
                         (var_name: 'stilt_df', var_type: pandas dataframe)
                      6. Start Date (var_name: "start_date", var_type: DateTime Object)
                      7. End Date (var_name: "end_date", var_type: DateTime Object)

    Output:           Bokeh Plot 
    
    """
    
    #import modules:
    import pandas as pd
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_Lx_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    #Loop through station obj list:
    for st in atm_stations:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station[2]:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station[2]
            station_info_dict['station_sampling_height'] = station[3]
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_Lx_ls: 
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_Lx_df = pd.concat(data_df_ls)
    
    #Add column with datetime object:
    data_Lx_df['DateTime'] = pd.to_datetime(data_Lx_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_Lx = data_Lx_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_Lx.sort_index(inplace=True)
    
    #Filter dataframe to only contain data for the selected year:
    data_Lx_df_filt = data_df_ind_Lx.loc[start_date:end_date]
    
    #Call plotting function:
    p = plot_stilt_icos_single_station_binary(data_Lx_df_filt,
                                              station_info_dict,
                                              tracer_info_dict,
                                              icos_data_level,
                                              stilt_df)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p)
    


In [ ]:
def update_icos_single_station_plot_binary(data_obj_id_Lx_ls, station_code, station_sampl_height, tracer, level, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 08 15:00:00 2019
    Last Changed:     Wed May 08 15:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that updates the content of an interactive Bokeh plot with ICOS Level-1 or
                      ICOS Level-2 data. The function gets the data object IDs of the new ICOS data objects and
                      calls functions to read their corresponding data files. Then the ICOS data are filtered
                      to only include information for the selected dates. The dataframes with the new ICOS data
                      are then sent to a plot function. The plot function returns a Bokeh Figure Object which
                      is plotted. The interactive plot was developed using the Bokeh interactive visualization
                      library. Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. List of data dataframes for every ICOS Level-x Data File
                         for a specific ICOS station and timeperiod.
                         (var_name: 'data_obj_id_L1_ls', var_type: List of pandas dataframes)
                      2. List with ICOS Station Info
                         (var_name: "station", var_type: List of Strings)
                      3. Tracer/gas (var_name: "tracer", var_type: String)
                      4. ICOS Data Level (var_name: "icos_data_level", var_type: Integer)
                      5. Dataframe with STILT Model Result for a specific ICOS Station
                         (var_name: 'stilt_df', var_type: pandas dataframe)
                      6. Start Date (var_name: "start_date", var_type: DateTime Object)
                      7. End Date (var_name: "end_date", var_type: DateTime Object)

    Output:           Bokeh Plot 
    
    """
    
    #import modules:
    import pandas as pd
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_Lx_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    #Loop through station obj list:
    for st in atm_stations:
        
        #Check if the station ID of the current station obj
        #is the same as the selected station ID
        if st.stationId==station_code:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station_code
            station_info_dict['station_sampling_height'] = station_sampl_height
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)

    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_Lx_ls: 
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_Lx_df = pd.concat(data_df_ls)
    
    #Add column with datetime object:
    data_Lx_df['DateTime'] = pd.to_datetime(data_Lx_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_Lx = data_Lx_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind_Lx.sort_index(inplace=True)
    
    
    #Filter dataframe to only contain data for the selected year:
    data_Lx_df_filt = data_df_ind_Lx.loc[start_date:end_date]
        
    #Call plotting function:
    p = plot_icos_single_station_binary(data_Lx_df_filt, station_info_dict, tracer_info_dict, level)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p)
   

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='footprint_funcs'></a>

<br>
<br>

## 7. STILT Footprint functions
This part includes functions that read STILT footprints and visualize them in static maps.

In [ ]:
def read_aggreg_footprints(station, date_range, stilt_icos_availability_df, pathFP, timeselect='all'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Nov 09 14:00:00 2018
    Last Changed:     Mon May 13 12:00:00 2018
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to read and aggregate footprints for given time range.
                      
    Input parameters: 1. STILT Station ID
                         (var_name: "station", var_type: String)
                      2. Set of Datetime Objects
                         (var_name: "date_range", var_type: Pandas Dataframe)
                      3. Dataframe with info about ICOS Atmospheric Stations with STILT results
                         (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)
                      4. Time Selection, available options: ('all', 'daytime', 'nighttime')
                         (var_name: "timeselect", var_type: String)

    Output:           1. Total number of Footprints available for the given station & date range
                         (var_name: "nfp", var_type: Integer)
                      2. Aggregated footprint
                         (var_name: "fp", var_type: Masked Array of Floats)
                      3. Longitudes
                         (var_name: "lon", var_type: Masked Array of Floats)
                      4. Latitudes
                         (var_name: "lat", var_type: Masked Array of Floats)
                      5. Title of plot
                         (var_name: "title", var_type: String)
    
    """
    
    #Create and initialize variables:
    fp=[]        #List to store footprints
    nfp=0        #Store total number of footprints
    first = True #Control variable -> cotrols 1st loop iteration
    
    # loop over all dates and read netcdf files
    for dd in date_range:
        filename=(pathFP+
                  stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==station].values[0]+'/'+
                  str(dd.year)+'/'+str(dd.month).zfill(2)+'/'+
                  str(dd.year)+'x'+str(dd.month).zfill(2)+'x'+
                  str(dd.day).zfill(2)+'x'+str(dd.hour).zfill(2)+'/foot')
        
        #If path to footprint exists, read netcdf-file:
        if os.path.isfile(filename):
            f_fp = cdf.Dataset(filename)
            
            #If this is the 1st iteration,
            #get footprint as well as latitudes & longitudes:
            if (first):
                fp=f_fp.variables['foot'][:,:,:]
                lon=f_fp.variables['lon'][:]
                lat=f_fp.variables['lat'][:]
                
                #Set variable controling check of 1st loop iteration to False:
                first = False
                
            #If this is not the first iteration of the loop:    
            else:
                
                #Read footprint:
                fp=fp+f_fp.variables['foot'][:,:,:]
            
            #Close file:
            f_fp.close()
            
            #Increase the counter of total num of footprints:
            nfp+=1        
    
    
    #If the total number of footprints is greater than zero:
    if nfp > 0:
        
        #Aggregate footprints:
        fp=fp/nfp
    

    #If no footprints are found:
    if(nfp==0):
        
        #Set output values:
        nfp = 0
        fp = None
        lon = None
        lat = None 
        title = ""
    
    
    #If there are footprints:
    else:
        #Create variable to store the footprint-map title:
        title = (date_range.min().strftime('%Y-%m-%d')+' - '+date_range.max().strftime('%Y-%m-%d')+'\n'+
                 'time selection: '+timeselect)

    #Return output:
    return nfp, fp, lon, lat, title

In [ ]:
def lonlat_2_ixjy(slon,slat,mlon,mlat):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon May 13 15:00:00 2019
    Last Changed:     Mon May 13 15:00:00 2019
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to convert station longitude and latitude (slat, slon) to indices
                      of STILT model grid (ix,jy)
                      
    Input parameters: 1. ICOS Station Longitude
                         (var_name: 'slon', var_type: Float)
                      2. ICOS Station Latitude
                         (var_name: "slat", var_type: Float)
                      3. Longitudes of the STILT Model Grid
                         (var_name: "mlon", var_type: Array of Floats)
                      4. Latitudes of the STILT Model Grid
                         (var_name: "mlat", var_type: Array of Floats)
                      

    Output:           Integer Variables (2)
    
    """
    
    #Get the STILT Model Grid-indices for the ICOS Station lat/lon:
    ix = (np.abs(mlon-slon)).argmin()
    jy = (np.abs(mlat-slat)).argmin()
    
    #Return STILT Model Grid indices:
    return ix,jy

In [ ]:
# function to plot maps (show station location if station is provided and zoom in second plot if zoom is provided)
def plot_fp_maps(field, lon, lat, station_info_df,
                 title='', label='', unit='', linlog='linear', station='',
                 vmin=None, vmax=None, colors='GnBu',midpoint=False):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon May 13 15:00:00 2019
    Last Changed:     Mon May 13 15:00:00 2019
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to plot footprint maps, depicting the station location. This function returns two
                      matplotlib plots with the footprints for a given ICOS Atmosphere Station for a given time
                      period. The first plot presents the STILT footprint over an area that covers the entire
                      European continent. The second plot presents the footprint over a smaller area around the
                      station.
                      
    Input parameters:  1. STILT Footprints for a specific ICOS Station
                          (var_name: 'field', var_type: Array)
                       2. Longitudes of the STILT Model Grid
                          (var_name: "lon", var_type: Array of Floats)
                       3. Latitudes of the STILT Model Grid
                          (var_name: "lat", var_type: Array of Floats)
                       4. Station info Dataframe
                          (var_name: "station_info_df", var_type: Pandas DataFrame)
                       5. Footprint Map Title
                          (var_name: "title", var_type: String)
                       6. Footprint Map Label
                          (var_name: "label", var_type: String)
                       7. Footprint Map Units
                          (var_name: "unit", var_type: String)
                       8. Colorscale Color Assignment Method
                          (var_name: "linlog", var_type: String)
                       9. List with Station Info
                          (var_name: "station", var_type: List of Strings)
                      10. Colorscale Lower Limit Footprint Value
                          (var_name: "vmin", var_type: Float)
                      11. Colorscale Upper Limit Footprint Value
                          (var_name: "vmax", var_type: Float)
                      12. Colorscale Name
                          (var_name: "colors", var_type: String)
                      13. Midpoint for diverging colorscales
                          (var_name: "midpoint", var_type: Float or Boolean)
                      

    Output:           Matplotlib Footprint Maps (if available)
    
    """
    
    
    #Check the dimensions of the Footprint array:
    if np.shape(field)[0] > 1:
        print ('More than one field: ',np.shape(field)[0],' Only the first will be plotted!!!')
    
    #Create figure and set figure size:
    fig = plt.figure(figsize=(15,8))

    # set up a map
    ax = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
    img_extent = (lon.min(), lon.max(), lat.min(), lat.max())
    #ax.set_extent([-15,35,33,72],crs=ccrs.PlateCarree())
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()],crs=ccrs.PlateCarree())
    #ax.gridlines(draw_labels=True)

    # Create a feature for Countries at 1:50m from Natural Earth
    countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    #ax.coastlines(resolution='50m', color='white', linewidth=0.3

    #cmap = p.get_cmap('Blues')
    #cmap = p.get_cmap('GnBu')
    cmap = plt.get_cmap(colors)
    if linlog == 'linear':
        print(' ')
        im=ax.imshow(field[0,:,:], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='neither')
        cbar.set_label(label+'  '+unit)
    
    else:
        im=ax.imshow(np.log10(field)[0,:,:], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.05, fraction=0.055,extend='neither')
        cbar.set_label(label+'  log$_{10}$ '+unit)
    plt.title(title)
    ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(field[0,:,:]), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(field[0,:,:]), horizontalalignment='right',transform=ax.transAxes)
    
   
    if station != '':
        #show station location if station is provided
        ax.plot(float(station_info_df.lon.loc[station_info_df['stationId']==station[2]].values[0]),
                float(station_info_df.lat.loc[station_info_df['stationId']==station[2]].values[0]),
                'm+',markersize=8,transform=ccrs.PlateCarree())
    #ax.plot(10.0,53.0,'m+',markersize=8,transform=ccrs.PlateCarree())
 


    #grid cell index of station 
    ix,jy = lonlat_2_ixjy(float(station_info_df.lon.loc[station_info_df['stationId']==station[2]].values[0]),
                          float(station_info_df.lat.loc[station_info_df['stationId']==station[2]].values[0]),
                          lon,lat)

    # define zoom area 
    i1 = np.max([ix-35,0])
    i2 = np.min([ix+35,400])
    j1 = np.max([jy-42,0])
    j2 = np.min([jy+42,480])

    # set up a map
    ax = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
    img_extent = (lon[i1:i2].min(), lon[i1:i2].max(), lat[j1:j2].min(), lat[j1:j2].max())
    #ax.set_extent([-15,35,33,72],crs=ccrs.PlateCarree())
    ax.set_extent([lon[i1:i2].min(), lon[i1:i2].max(), lat[j1:j2].min(), lat[j1:j2].max()],crs=ccrs.PlateCarree())
    #ax.gridlines(draw_labels=True)

    # Create a feature for Countries at 1:50m from Natural Earth
    countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    #ax.coastlines(resolution='50m', color='white', linewidth=0.3

    if linlog == 'linear':
        print(' ')
        im=ax.imshow(field[0,j1:j2,i1:i2], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='neither')
        cbar.set_label(label+'  '+unit)
        
    else:
        
        im=ax.imshow(np.log10(field)[0,j1:j2,i1:i2], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.05,fraction=0.055,extend='neither')
        cbar.set_label(label+'  log$_{10}$ '+unit)
    
    plt.title(title)
    ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(field[0,j1:j2,i1:i2]), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(field[0,j1:j2,i1:i2]), horizontalalignment='right',transform=ax.transAxes)
        
    if station != '':
        
        #show station location if station is provided
        ax.plot(float(station_info_df.lon.loc[station_info_df['stationId']==station[2]].values[0]),
                float(station_info_df.lat.loc[station_info_df['stationId']==station[2]].values[0]),
                'm+',markersize=8,transform=ccrs.PlateCarree())
        
    #Show plots:
    plt.show()
    plt.close()
   

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='widget_func'></a>

<br>
<br>

## 8. Widget functions
This part includes functions that create widget forms. Widget are interactive elements like dropdown lists, checkboxes, etc. in Python.

In [ ]:
#Create function for ICOS AS map widget form:
def map_wdgt_form_icos_stilt(icos_stilt_df, station_info_df):
    
    #Import modules:
    from ipywidgets import VBox, Button, Dropdown, Output, Label
    from IPython.display import clear_output
    
    #Create dropdown list for ICOS stations:
    stations_wdgt =  Dropdown(options = sorted([(station_info_df['name'].iloc[i],
                                                 station_info_df['stationId'].iloc[i])
                                                for i in range(len(station_info_df))]), 
                              description='Station')
    
    #Create dropdown list basemap options:
    basemap_wdgt = Dropdown(options = ['Imagery', 'OpenStreetMap'], description='Basemap')
    
    #Create button widget (execution):
    button_exe = Button(description='Update map',
                        disabled=False,
                        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip='Press the button to update map',
                        icon='check')
    
    
    #Format widgets:
    stations_wdgt.layout.width = '379px'
    stations_wdgt.style.description_width = 'initial'
    stations_wdgt.layout.margin = '2px 2px 2px 238px'
    basemap_wdgt.layout.width = '394px'
    basemap_wdgt.style.description_width = 'initial'
    basemap_wdgt.layout.margin = '2px 2px 2px 224px'
    button_exe.style.button_color = '#3973ac'
    button_exe.layout.width = '250px'
    button_exe.layout.margin = '50px 100px 40px 330px'
    
    #Create form object:
    form_out = Output()

    #Create output object:
    plot_out = Output()
    
    #Function that executes on button_click (calculate score):
    def on_exe_bttn_clicked(button_c):
    
        #Open output object:
        with plot_out:

            #Delete previous output:
            clear_output()

            #Show new map:
            plotmap_stilt(st_info_df, stations_wdgt.value, basemap_wdgt.value, d_icon='cloud', icon_col='orange')
            
    #Call function on button_click-event (calculate score):
    button_exe.on_click(on_exe_bttn_clicked)

    #Open output obj:a
    with form_out:

        #Clean previous values:
        clear_output()

        #Show plot:
        display(VBox([stations_wdgt, basemap_wdgt, button_exe, plot_out]))

    #Display form:
    display(form_out)

In [ ]:
def create_widgets_stilt_icos_plots(stilt_icos_availability_df, station_info_df, path_stilt):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 14:00:00 2019
    Last Changed:     Wed Oct 14 14:00:00 2020
    Version:          1.1.0
    Author(s):        Karolina Pantazatou
    
    Description:      Function that creates a set of widgets; a station dropdown list, a year dropdown list,
                      a citation checkbox and a button, populates the dropdown lists with values, captures the 
                      user's input and calls a function to update the contents of the plot. If the citation checkbox
                      is checked, a string including the citation text for all data layers included in the plot
                      will be displayed.
                      
    Input parameters: 1. Dataframe with info about ICOS Atmospheric Stations with STILT results
                         (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)
                      2. Dataframe with info about ICOS stations for which STILT results are available
                         (var_name: 'station_info_df', var_type: Pandas Dataframe)
                      3. Path to location where STILT results are stored 
                         (var_name: 'path_stilt', var_type: String)

    Output:           Plot and/or Citation and/or Warning Message
    
    """
    
    #Get STILT availability dataframe:
    stilt_avail_df = get_stilt_availability_df(path_stilt, stilt_icos_availability_df, 'stiltStationId')
    
    #Get a list of labels of ICOS Atmospheric stations for which STILT model results are available:
    init_station_ls = create_icos_stilt_labels(stilt_icos_availability_df, station_info_df)

    #Remove test-result for SMEAR:
    stations_updated = [x for x in init_station_ls if x!=('SMEAR II-ICOS Hyytiälä (alt. 127.0)', ['SMR127', '40.0', 'SMR', '125.0'])]

    #Create widgets:
    station =  Dropdown(options = stations_updated,
                        description='Station')
    year = Dropdown(options = np.arange(2016, datetime.now().year +1, 1).tolist())

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Station, Year, Citation):
        
        #Define start date:
        start_date = pd.Timestamp(Year,1,1,0)
        
        #Define end date:
        end_date = pd.Timestamp(Year+1,1,1,0)
        
        #Create a pandas dataframe containing one column of datetime objects with 3-hour intervals:
        date_range = pd.date_range(start_date, end_date, freq='3H')
        

        #Check availability of STILT-data (if = False --- > data is available, True ---> data is not available)
        stilt_availability = stilt_avail_df.loc[(stilt_avail_df['stiltStationId']==Station[0]) &
                                                (stilt_avail_df['year']==Year) &
                                                (stilt_avail_df.month>0)].empty
        
        #Check availability of ICOS-L1-data:
        icos_L1_availability = stilt_icos_availability_df.loc[(stilt_icos_availability_df['stiltStationId']==Station[0]) &
                                                              (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series') & 
                                                              (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')) &
                                                                (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')))|
                                                               (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(year=Year, month=12, day=31, hour=23, tz='UTC'))&
                                                                 (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')))))].empty
        
        #Check availability of ICOS-L2-data:
        icos_L2_availability = stilt_icos_availability_df.loc[(stilt_icos_availability_df['stiltStationId']==Station[0]) &
                                                              (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release') & 
                                                              (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')) &
                                                                (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')))|
                                                               (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(year=Year, month=12, day=31, hour=23, tz='UTC'))&
                                                                 (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(year=Year, month=1, day=1, hour=0, tz='UTC')))))].empty
        
        
        #If no STILT-data, ICOS-L1-data or ICOS-L2-data are available:
        if((stilt_availability) & (icos_L1_availability) & (icos_L2_availability)):
            
            #Print message:
            print("\033[0;31;1m No ICOS Level-1 or Level-2 data available.\n No STILT results available yet.\n\n")
                   
                
                    
        #If only STILT-data are available:    
        elif(((icos_L1_availability) & (icos_L2_availability)) & (stilt_availability==False)):
                    
            #Print message:
            print("\033[0;31;1m No ICOS Level-1 or Level-2 data available.\n\n")

            #Get STILT-data:
            stilt_df = read_stilt_timeseries(Station[0], date_range, stilt_icos_availability_df, path_stilt)
            
            #Plot STILT-data:
            plot_stilt_single_station_binary(stilt_df, Station, 'co2')
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Print citation:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")
   

                    
        #If only ICOS data are available:
        elif(((icos_L1_availability==False) & (icos_L2_availability==False)) & (stilt_availability)):
                    
            #Print message:
            print("\033[0;31;1m No STILT-data available yet.\n\n")
                    
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                          (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                          (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series')].values)
            
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_L2_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                          (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                          (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release')].values)
            
            #If Level-1 & Level-2 data are available for the selected tracer and station:
            if((len(data_obj_url_L1_ls)>0) & (len(data_obj_url_L2_ls)>0)):
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                     for j in range(len(data_obj_url_L1_ls))]
                
                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_L2_ls[k].replace('https://meta.icos-cp.eu/objects/', '')
                                     for k in range(len(data_obj_url_L2_ls))]
                
                #Call function to return plot for the selected station (Level 1 & 2 Data):
                update_icos_single_station_plot_L1_L2_binary(data_obj_id_L1_ls,
                                                             data_obj_id_L2_ls,
                                                             Station[2],
                                                             Station[3],
                                                             'co2',
                                                             start_date,
                                                             end_date)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]

                    #Get a list with citation info for every ICOS Level-2 data object:
                    cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L2_ls]

                    #Get a list with citation info for every ICOS Level-1 and Level-2 data object.
                    #Concatenate citation lists to one list: 
                    cit_ls = cit_ls_L1 + cit_ls_L2
                    
                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
                 
                
            #If no L1-data or L2-data are available for the selected tracer and station:
            else:
                print("\033[0;31;1m No Level-1 or Level-2 data available for the selected tracer and station at present.\n Try a new combination!\n\n")
                
                   
                    
        #If only ICOS Level-1 data are available:
        elif(((icos_L1_availability==False) & (icos_L2_availability)) & (stilt_availability)):
                    
            #Print message:
            print("\033[0;31;1m No ICOS Level-2 or STILT data are available yet.\n\n")
                    
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                          (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                          (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series')].values)
            
            #If L1-data is available for the selected tracer and station:
            if(len(data_obj_url_L1_ls)>0):
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[i].replace('https://meta.icos-cp.eu/objects/', '') 
                                     for i in range(len(data_obj_url_L1_ls))]
    
                #Call function to return plot for the selected station (Level 1 Data):
                update_icos_single_station_plot_binary(data_obj_id_L1_ls,
                                                       Station[2],
                                                       Station[3],
                                                       'co2',
                                                       1,
                                                       start_date,
                                                       end_date)
            
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]
                    
                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls_L1:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If no L1-data is available for the selected tracer and station:
            else:
                print("\033[0;31;1m "+ 'No '+tracer.upper().translate(SUB)+
                      ' Level-1 data available for the selected station.\n\n')
                    
                
                
        #If only ICOS Level-2 data are available:
        elif(((icos_L1_availability) & (icos_L2_availability==False)) & (stilt_availability)):
                    
            #Print message:
            print("\033[0;31;1m No ICOS Level-1 or STILT data are available yet.\n\n")
                    
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                       (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                       (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release')].values)
            
            #If L2-data is available for the selected tracer and station:
            if(len(data_obj_url_ls)>0):

                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '')
                                     for i in range(len(data_obj_url_ls))]

                #Call function to return plot and map for the selected station:
                update_icos_single_station_plot_binary(data_obj_id_L2_ls,
                                                       Station[2],
                                                       Station[3], 
                                                       'co2',
                                                       2,
                                                       start_date,
                                                       end_date)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_ls]
                    
                    
                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls_L2:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
      

            #If no L2-data are available for the selected tracer and station:
            else:
                print("\033[0;31;1m " +'No '+tracer.upper().translate(SUB)+' Level-2 data available for the selected station.\n\n')
               
                
                    
        #If only STILT and ICOS Level-1 data are available:
        elif(((icos_L1_availability==False) & (icos_L2_availability)) & (stilt_availability==False)):
                    
            #Print message:
            print("\033[0;31;1m No ICOS Level-2 data are available.\n\n")
            
            #Get STILT-data:
            stilt_df = read_stilt_timeseries(Station[0], date_range, stilt_icos_availability_df, path_stilt)
            
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                       (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                       (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series')].values)
          
            #If L1-data is available for the selected tracer and station:
            if(len(data_obj_url_ls)>0):

                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '')
                                     for i in range(len(data_obj_url_ls))]
                
                #Plot ICOS Level-1 & STILT-data:
                update_icos_stilt_single_station_plot_binary(data_obj_id_L1_ls,
                                                             Station,
                                                             'co2',
                                                             2,
                                                             stilt_df,
                                                             start_date,
                                                             end_date)
                
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_ls]
                    
                    #Print citation title:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls_L1:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")

                    #Print STILT citation:
                    printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")
                

            #If no L2-data are available for the selected tracer and station:
            else:
                print("\033[0;31;1m "+'No '+tracer.upper().translate(SUB)+' Level-1 data available for the selected station.\n\n')
            
            
          
        #If only STILT and ICOS Level-2 data are available:
        elif(((icos_L1_availability) & (icos_L2_availability==False)) & (stilt_availability==False)):
                    
            #Print message:
            print("\033[0;31;1m No ICOS Level-1 data are available.\n\n")
            
            #Get STILT-data:
            stilt_df = read_stilt_timeseries(Station[0], date_range, stilt_icos_availability_df, path_stilt)
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                       (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                       (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release')].values)
            
            #If L2-data is available for the selected tracer and station:
            if(len(data_obj_url_ls)>0):

                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '')
                                  for i in range(len(data_obj_url_ls))]
                
                #Plot ICOS Level-2 & STILT-data:
                update_icos_stilt_single_station_plot_binary(data_obj_id_L2_ls,
                                                             Station,
                                                             'co2',
                                                             2,
                                                             stilt_df,
                                                             start_date,
                                                             end_date)
                
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_ls]
                    
                    #Print citation title:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls_L2:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")

                    #Print STILT citation:
                    printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")
                

            #If no L2-data are available for the selected tracer and station:
            else:
                print("\033[0;31;1m "+'No '+tracer.upper().translate(SUB)+' Level-2 data available for the selected station.\n\n')
            
     
        #If STILT and ICOS Level-2 and ICOS Level-1 data are available:
        elif(((icos_L1_availability==False) & (icos_L2_availability==False)) & (stilt_availability==False)):
                    
            #Print message:
            print("\033[0;31;1m Data available.\n\n")
            
            #Get STILT-data:
            stilt_df = read_stilt_timeseries(Station[0], date_range, stilt_icos_availability_df, path_stilt)
            
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                          (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                          (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series')].values)
            
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_L2_ls = list(stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                          (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                          (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release')].values)
            
            #If Level-1 & Level-2 data are available for the selected tracer and station:
            if((len(data_obj_url_L1_ls)>0) & (len(data_obj_url_L2_ls)>0)):
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                     for j in range(len(data_obj_url_L1_ls))]
                
                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_L2_ls[k].replace('https://meta.icos-cp.eu/objects/', '')
                                     for k in range(len(data_obj_url_L2_ls))]
                
                #Call function to return plot for the selected station (Level 1 & 2 Data):
                update_icos_L1_L2_stilt_single_station_plot_binary(stilt_df,
                                                            data_obj_id_L1_ls,
                                                            data_obj_id_L2_ls,
                                                            Station,
                                                            'co2',
                                                            start_date,
                                                            end_date)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L1_ls]

                    #Get a list with citation info for every ICOS Level-2 data object:
                    cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                 for dobj in data_obj_url_L2_ls]

                    #Get a list with citation info for every ICOS Level-1 and Level-2 data object.
                    #Concatenate citation lists to one list: 
                    cit_ls = cit_ls_L1 + cit_ls_L2
                    
                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                    #Loop through all citations:
                    for cit in cit_ls:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
                        
                    #Print STILT citation:
                    printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")
                 
                
            #If no L1-data or L2-data are available for the selected tracer and station:
            else:
                print("\033[0;31;1m No Level-1 or Level-2 data available for the selected tracer and station at present.\n Try a new combination!\n\n")
 


        else:
            
            #Print message:
            print("\033[0;31;1m No data available!\n\n")
                    
    

    

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Station=station,
                                 Year = year,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def create_widgets_stilt_icos_fp(stilt_icos_availability_df, station_info_df, path_stilt):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Sat May 11 14:00:00 2019
    Last Changed:     Sat May 11 14:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station dropdown list, two datepickers, 
                      two checkboxes and a button, populates the dropdown lists with values, captures the 
                      user's input and calls a function to update the contents of the plot. The function
                      also outputs a set of Footprint maps and data citation strings, if the corresponding
                      checkboxes are checked.
                      
    Input parameters: Dataframe with info about ICOS Atmospheric Stations with STILT results
                      (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)

    Output:           Plot with Map or Warning Message
    
    """

    #Get STILT availability dataframe:
    stilt_avail_df = get_stilt_availability_df(path_stilt, stilt_icos_availability_df, 'stiltStationId')
    
    #Get a list of labels of ICOS Atmospheric stations for which STILT model results are available:
    init_station_ls = create_icos_stilt_labels(stilt_icos_availability_df, station_info_df)

    #Remove test-result for SMEAR:
    stations_updated = [x for x in init_station_ls if x!=('SMEAR II-ICOS Hyytiälä (alt. 127.0)', ['SMR127', '40.0', 'SMR', '125.0'])]

    #Create widgets:
    station =  Dropdown(options = stations_updated,
                        description='Station')


    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Station, start_date, end_date, Footprint, Citation):
        
        #Check if either or both of the selected start_date or end_date are equal to NULL:
        if((start_date==None)|(end_date==None)):
            
            #Print message:
            print('\033[0;31;1m No date(s) selected!\n\n')
            
        else:
            #Compute the difference between end_date and start_date:
            diff = end_date - start_date
            
            #If start_date corresponds to a later date than end_date:
            if(diff.days<0):
                
                #Print message:
                print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')
            
            else:
                
                #Create a pandas dataframe containing one column of datetime objects with 3-hour intervals:
                date_range = pd.date_range(start_date, end_date+dt.timedelta(hours=24), freq='3H')
                
                #Get STILT-data:
                stilt_df = read_stilt_timeseries(Station[0], date_range, stilt_icos_availability_df, path_stilt)
                
                #Get a list of available ICOS Level-1 Data Object URLs:
                data_obj_url_L1_ls = stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                         (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                         (stilt_icos_availability_df['specLabel']=='ICOS ATC NRT CO2 growing time series')&
                                                                         (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(start_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(start_date, tz='UTC')))|
                                                                          ((stilt_icos_availability_df['timeStart']<=pd.Timestamp(end_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(end_date, tz='UTC')))|
                                                                          ((stilt_icos_availability_df['timeStart']>=pd.Timestamp(start_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']<=pd.Timestamp(end_date, tz='UTC'))))].values
                
                #Get a list of available ICOS Level-2 Data Object URLs:
                data_obj_url_L2_ls = stilt_icos_availability_df.dobj.loc[(stilt_icos_availability_df['stationId']==Station[2]) &
                                                                         (stilt_icos_availability_df['samplingheight']==Station[3]) &
                                                                         (stilt_icos_availability_df['specLabel']=='ICOS ATC CO2 Release')&
                                                                         (((stilt_icos_availability_df['timeStart']<=pd.Timestamp(start_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(start_date, tz='UTC')))|
                                                                          ((stilt_icos_availability_df['timeStart']<=pd.Timestamp(end_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']>=pd.Timestamp(end_date, tz='UTC')))|
                                                                          ((stilt_icos_availability_df['timeStart']>=pd.Timestamp(start_date, tz='UTC'))&
                                                                           (stilt_icos_availability_df['timeEnd']<=pd.Timestamp(end_date, tz='UTC'))))].values
                
                
                
                #If only STILT model results are available:
                if((data_obj_url_L1_ls.size==0) & (data_obj_url_L2_ls.size==0) & (stilt_df.size>0)):
                    
                    #Print message:
                    print("\033[0;31;1m No ICOS Level-1 or Level-2 data available.\033[0;31;0m\n\n")

                    #Plot STILT-data:
                    plot_stilt_single_station_binary(stilt_df, Station, 'co2')
                        
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Get footprint data and plot map:
                        stilt_nfp, stilt_fp, stilt_lon, stilt_lat, stilt_title = read_aggreg_footprints(Station[0],
                                                                                                        date_range,
                                                                                                        stilt_icos_availability_df,
                                                                                                        path_stilt)
                        
                        #If no footprints are found:
                        if(stilt_nfp == 0):
                            
                            #Print message:
                            print("\033[0;31;1m No footprints available.\033[0;31;0m\n\n")
                      
                    
                        #If footprints are found:
                        else:
                            
                            #Set footprints that are equal to zero, equal to NaN:
                            stilt_fp.data[stilt_fp.data==0.0] = np.nan
                            
                            #Call function to plot footprint maps:
                            plot_fp_maps(stilt_fp, stilt_lon, stilt_lat, station_info_df,
                                         title='Aggregated Footrpint (n='+str(stilt_nfp)+')\n'+stilt_title+
                                         '\nStation: '+
                                         station_info_df.name.loc[station_info_df.stationId==Station[2]].values[0]+
                                         ' ('+Station[0] + ') ' + Station[1] + ' m agl ' + '\n['+
                                         stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==Station[0]].values[0]+
                                         ']',
                                         label='Surface Influence',
                                         unit='[ppm / (\u03BCmol / m\u00B2s)]',
                                         linlog="log10", station=Station, midpoint=True)
                  
                
                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Print citation:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                        printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")


                
                #If only ICOS Level-1 & Level-2 data are available for the selected tracer and station:
                elif((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size>0) & (stilt_df.size==0)):
                    
                    #Print message:
                    print("\033[0;31;1m No STILT model results available yet ...\033[0;31;0m\n\n")

                    #Get a list of data object IDs (L1-data):
                    data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                         for j in range(data_obj_url_L1_ls.size)]

                    #Get a list of data object IDs (L2-data):
                    data_obj_id_L2_ls = [data_obj_url_L2_ls[k].replace('https://meta.icos-cp.eu/objects/', '')
                                         for k in range(data_obj_url_L2_ls.size)]

                    #Call function to return plot for the selected station (ICOS Level-1 & Level-2 Data):
                    update_icos_single_station_plot_L1_L2_binary (data_obj_id_L1_ls,
                                                                  data_obj_id_L2_ls,
                                                                  Station[2],
                                                                  Station[3],
                                                                  'co2',
                                                                  start_date,
                                                                  end_date)
                    
                    #Fix "update_icos_L1_L2_stilt_single_station_plot" function to filter ICOS data after dates!!!
                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Print:
                        print('\033[0;31;1m No STILT footprints available yet ...\033[0;31;0m\n\n')
                        
                        
                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L1_ls]

                        #Get a list with citation info for every ICOS Level-2 data object:
                        cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L2_ls]

                        #Concatenate citation lists to one list: 
                        cit_ls = cit_ls_L1 + cit_ls_L2

                        #Print citation:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")
                
                          
                          
                #If only ICOS Level-1 data are available for the selected tracer and station:
                elif((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size==0) & (stilt_df.size==0)):
                          
                    #Print message:
                    print("\033[0;31;1m No STILT model results or ICOS Level-2 data available.\033[0;31;0m\n\n")
                          
                    #Get a list of data object IDs (L1-data):
                    data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                         for j in range(data_obj_url_L1_ls.size)]
                    
                    #Call function to return plot for the selected station (ICOS Level-1 Data):
                    update_icos_single_station_plot_binary(data_obj_id_L1_ls,
                                                           Station[2],
                                                           Station[3],
                                                           'co2',
                                                           1,
                                                           start_date,
                                                           end_date)
                    
                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Print:
                        print('\033[0;31;1m No STILT footprints available yet ...\033[0;31;0m\n\n')
                        
                        
                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L1 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L1_ls]

                        #Print citation:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls_L1:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")
                    
                    
                    
                #If only ICOS Level-2 data are available for the selected tracer and station:
                elif((data_obj_url_L1_ls.size==0) & (data_obj_url_L2_ls.size>0) & (stilt_df.size==0)):
                    
                    #Print message:
                    print("\033[0;31;1m No STILT model results or ICOS Level-1 data available.\033[0;31;0m\n\n")
                    
                    #Get a list of data object IDs (L1-data):
                    data_obj_id_L2_ls = [data_obj_url_L2_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                         for j in range(data_obj_url_L2_ls.size)]
                    
                    #Call function to return plot for the selected station (ICOS Level-2 Data):
                    update_icos_single_station_plot_binary(data_obj_id_L2_ls,
                                                           Station[2],
                                                           Station[3],
                                                           'co2',
                                                           2,
                                                           start_date,
                                                           end_date)
                    
                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Print:
                        print('\033[0;31;1m No STILT footprints available yet ...\033[0;31;0m\n\n')
                        
                        
                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L2_ls]

                        #Print citation:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls_L2:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")
                    
                    
                    
                #If only STILT Model results & ICOS Level-1 data are available for the selected tracer and station:
                elif((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size==0) & (stilt_df.size>0)):
                    
                    #Print message:
                    print("\033[0;31;1m No ICOS Level-2 data available.\033[0;31;0m\n\n")
                    
                    #Get a list of data object IDs (L1-data):
                    data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                         for j in range(data_obj_url_L1_ls.size)]
                
                    #Plot ICOS Level-2 & STILT-data:
                    update_icos_stilt_single_station_plot_binary(data_obj_id_L1_ls,
                                                                 Station,
                                                                 'co2',
                                                                 2,
                                                                 stilt_df,
                                                                 start_date,
                                                                 end_date)
                    
                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Get footprint data and plot map:
                        stilt_nfp, stilt_fp, stilt_lon, stilt_lat, stilt_title = read_aggreg_footprints(Station[0],
                                                                                                        date_range,
                                                                                                        stilt_icos_availability_df,
                                                                                                        path_stilt)
                        
                        #If no footprints are found:
                        if(stilt_nfp == 0):
                            
                            #Print message:
                            print("\033[0;31;1m No footprints available.\033[0;31;0m\n\n")
                      
                    
                        #If footprints are found:
                        else:
                            
                            #Set footprints that are equal to zero, equal to NaN:
                            stilt_fp.data[stilt_fp.data==0.0] = np.nan
                            
                            #Call function to plot footprint maps:
                            plot_fp_maps(stilt_fp, stilt_lon, stilt_lat, station_info_df,
                                         title='Aggregated Footrpint (n='+str(stilt_nfp)+')\n'+stilt_title+
                                         '\nStation: '+
                                         station_info_df.name.loc[station_info_df.stationId==Station[2]].values[0]+
                                         ' ('+Station[0] + ') ' + Station[1] + ' m agl ' + '\n['+
                                         stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==Station[0]].values[0]+
                                         ']',
                                         label='Surface Influence',
                                         unit='[ppm / (\u03BCmol / m\u00B2s)]',
                                         linlog="log10", station=Station, midpoint=True)
                  


                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L1_ls]

                        #Print citation title:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls_L1:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")

                        #Print STILT citation:
                        printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")

                    
                    
                #If only STILT Model results & ICOS Level-2 data are available for the selected tracer and station:
                elif((data_obj_url_L1_ls.size==0) & (data_obj_url_L2_ls.size>0) & (stilt_df.size>0)):
                    
                    #Print message:
                    print("\033[0;31;1m No ICOS Level-1 data available.\n\n\033[0;31;0m")
                    
                    #Get a list of data object IDs (L2-data):
                    data_obj_id_L2_ls = [data_obj_url_L2_ls[i].replace('https://meta.icos-cp.eu/objects/', '')
                                         for i in range(data_obj_url_L2_ls.size)]

                    #Plot ICOS Level-2 & STILT-data:
                    update_icos_stilt_single_station_plot_binary(data_obj_id_L2_ls,
                                                                 Station,
                                                                 'co2',
                                                                 2,
                                                                 stilt_df,
                                                                 start_date, 
                                                                 end_date)
                    
                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Get footprint data and plot map:
                        stilt_nfp, stilt_fp, stilt_lon, stilt_lat, stilt_title = read_aggreg_footprints(Station[0],
                                                                                                        date_range,
                                                                                                        stilt_icos_availability_df,
                                                                                                        path_stilt)
                        #If no footprints are found:
                        if(stilt_nfp == 0):
                            
                            #Print message:
                            print("\033[0;31;1m No footprints available.\033[0;31;0m\n\n")
                      
                    
                        #If footprints are found:
                        else:
                            
                            #Set footprints that are equal to zero, equal to NaN:
                            stilt_fp.data[stilt_fp.data==0.0] = np.nan
                            
                            #Call function to plot footprint maps:
                            plot_fp_maps(stilt_fp, stilt_lon, stilt_lat, station_info_df,
                                         title='Aggregated Footrpint (n='+str(stilt_nfp)+')\n'+stilt_title+
                                         '\nStation: '+
                                         station_info_df.name.loc[station_info_df.stationId==Station[2]].values[0]+
                                         ' ('+Station[0] + ') ' + Station[1] + ' m agl ' + '\n['+
                                         stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==Station[0]].values[0]+
                                         ']',
                                         label='Surface Influence',
                                         unit='[ppm / (\u03BCmol / m\u00B2s)]',
                                         linlog="log10", station=Station, midpoint=True)
                  

                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L2_ls]

                        #Print citation title:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls_L2:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")

                        #Print STILT citation:
                        printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")


                    
                    
                #If STILT Model results, ICOS Level-1 & Level-2 data are available:
                elif((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size>0) & (stilt_df.size>0)):
                    
                    #Print message:
                    #print("\033[0;31;1m Data available.\033[0;31;0m\n\n")
                    
                    #Get a list of data object IDs (L1-data):
                    data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                         for j in range(data_obj_url_L1_ls.size)]

                    #Get a list of data object IDs (L2-data):
                    data_obj_id_L2_ls = [data_obj_url_L2_ls[k].replace('https://meta.icos-cp.eu/objects/', '')
                                         for k in range(data_obj_url_L2_ls.size)]

                    #Call function to return plot for the selected station (Level-1 & Level-2 Data):
                    update_icos_L1_L2_stilt_single_station_plot_binary(stilt_df,
                                                                       data_obj_id_L1_ls,
                                                                       data_obj_id_L2_ls,
                                                                       Station,
                                                                       'co2',
                                                                       start_date,
                                                                       end_date)
                                    
                    #If the "footprint" checkbox is checked:
                    if(Footprint):
                        
                        #Get footprint data and plot map:
                        stilt_nfp, stilt_fp, stilt_lon, stilt_lat, stilt_title = read_aggreg_footprints(Station[0],
                                                                                                        date_range,
                                                                                                        stilt_icos_availability_df,
                                                                                                        path_stilt)
                        ##If no footprints are found:
                        if(stilt_nfp == 0):
                            
                            #Print message:
                            print("\033[0;31;1m No footprints available.\033[0;31;0m\n\n")
                      
                    
                        #If footprints are found:
                        else:
                            
                            #Set footprints that are equal to zero, equal to NaN:
                            stilt_fp.data[stilt_fp.data==0.0] = np.nan
                            
                            #Call function to plot footprint maps:
                            plot_fp_maps(stilt_fp, stilt_lon, stilt_lat, station_info_df,
                                         title='Aggregated Footrpint (n='+str(stilt_nfp)+')\n'+stilt_title+
                                         '\nStation: '+
                                         station_info_df.name.loc[station_info_df.stationId==Station[2]].values[0]+
                                         ' ('+Station[0] + ') ' + Station[1] + ' m agl ' + '\n['+
                                         stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==Station[0]].values[0]+
                                         ']',
                                         label='Surface Influence',
                                         unit='[ppm / (\u03BCmol / m\u00B2s)]',
                                         linlog="log10", station=Station, midpoint=True)
                  

                    #If the "citation" checkbox is checked:
                    if(Citation):

                        #Get a list with citation info for every ICOS Level-1 data object:
                        cit_ls_L1 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L1_ls]

                        #Get a list with citation info for every ICOS Level-2 data object:
                        cit_ls_L2 = [runsparql.RunSparql(sparql_query=sparqls.get_icos_citation(dobj), output_format='pandas').run().cit.iloc[0]
                                     for dobj in data_obj_url_L2_ls]

                        #Concatenate citation lists to one list: 
                        cit_ls = cit_ls_L1 + cit_ls_L2

                        #Print citation:
                        print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                        #Loop through all citations:
                        for cit in cit_ls:

                            #Print data object citation:
                            printmd("<sub>"+cit+"</sub>")

                        #Print STILT citation:
                        printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")


              
                #If no STILT Model results, ICOS Level-1 & Level-2 data are available:    
                else:
                    
                    #Print message:
                    print("\033[0;31;1m No STILT model results or ICOS data available.\033[0;31;0m\n\n")
                
            
            
            
            
            
            
            
            
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Station=station,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False),
                                 Footprint=Checkbox(value=False, description='Footprint', disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))
                                 #color=ColorPicker(concise=False,
                                                   #description='Pick a color',
                                                   #value='#3973ac',
                                                   #disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '10px 2px 2px 2px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left
            

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='main'></a>

<br>
<br>

## 9. Main function
Set of functions that get the main information of ICOS data and STILT results. The variables that store the output of the aforementioned function calls are global.


In [ ]:
#Get ICOS atmosphere stations:
atm_stations = station.getList(['as'])

In [ ]:
#Call function to get an icos-stilt availability dataframe:
availability_df = get_icos_stilt_availability_df(path_stiltweb_stations)

In [ ]:
availability_df

In [ ]:
#Get a dataframe with station info for all ICOS atmospheric stations:
icos_st_info_df = pd.concat([pd.DataFrame(atm_stations[i].info()) for i in range(len(atm_stations))])

#Filter out stations for which there are no STILT-results:
st_info_df = icos_st_info_df[icos_st_info_df['stationId'].isin(availability_df['stationId'].unique())].reset_index().drop('index', axis=1)


<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>